# Build a strategy from scratch

Welcome to the third part of the Flower federated learning tutorial. In previous parts of this tutorial, we introduced federated learning with PyTorch and Flower ([part 1](https://flower.ai/docs/framework/tutorial-get-started-with-flower-pytorch.html)) and we learned how strategies can be used to customize the execution on both the server and the clients ([part 2](https://flower.ai/docs/framework/tutorial-use-a-federated-learning-strategy-pytorch.html)).

In this notebook, we'll continue to customize the federated learning system we built previously by creating a custom version of FedAvg (again, using [Flower](https://flower.ai/) and [PyTorch](https://pytorch.org/)).

> [Star Flower on GitHub](https://github.com/adap/flower) ⭐️ and join the Flower community on Slack to connect, ask questions, and get help: [Join Slack](https://flower.ai/join-slack) 🌼 We'd love to hear from you in the `#introductions` channel! And if anything is unclear, head over to the `#questions` channel.

Let's build a new `Strategy` from scratch!

## Preparation

Before we begin with the actual code, let's make sure that we have everything we need.

### Installing dependencies

First, we install the necessary packages:

In [ ]:
!pip install -q flwr[simulation] torch torchvision

Now that we have all dependencies installed, we can import everything we need for this tutorial:

In [30]:
from collections import OrderedDict
from typing import Dict, List, Optional, Tuple

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10
import csv
import flwr as fl

DEVICE = torch.device("cuda")  # Try "cuda" to train on GPU
print(
    f"Training on {DEVICE} using PyTorch {torch.__version__} and Flower {fl.__version__}"
)

Training on cuda using PyTorch 1.13.1 and Flower 1.4.0


It is possible to switch to a runtime that has GPU acceleration enabled (on Google Colab: `Runtime > Change runtime type > Hardware acclerator: GPU > Save`). Note, however, that Google Colab is not always able to offer GPU acceleration. If you see an error related to GPU availability in one of the following sections, consider switching back to CPU-based execution by setting `DEVICE = torch.device("cpu")`. If the runtime has GPU acceleration enabled, you should see the output `Training on cuda`, otherwise it'll say `Training on cpu`.

### Data loading

Let's now load the CIFAR-10 training and test set, partition them into ten smaller datasets (each split into training and validation set), and wrap everything in their own `DataLoader`. We introduce a new parameter `num_clients` which allows us to call `load_datasets` with different numbers of clients.

In [2]:
NUM_CLIENTS = 10


def load_datasets(num_clients: int):
    # Download and transform CIFAR-10 (train and test)
    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )
    trainset = CIFAR10("./dataset", train=True, download=True, transform=transform)
    testset = CIFAR10("./dataset", train=False, download=True, transform=transform)

    # Split training set into `num_clients` partitions to simulate different local datasets
    partition_size = len(trainset) // num_clients
    lengths = [partition_size] * num_clients
    datasets = random_split(trainset, lengths, torch.Generator().manual_seed(42))

    # Split each partition into train/val and create DataLoader
    trainloaders = []
    valloaders = []
    for ds in datasets:
        len_val = len(ds) // 10  # 10 % validation set
        len_train = len(ds) - len_val
        lengths = [len_train, len_val]
        ds_train, ds_val = random_split(ds, lengths, torch.Generator().manual_seed(42))
        trainloaders.append(DataLoader(ds_train, batch_size=32, shuffle=True))
        valloaders.append(DataLoader(ds_val, batch_size=32))
    testloader = DataLoader(testset, batch_size=32)
    return trainloaders, valloaders, testloader


trainloaders, valloaders, testloader = load_datasets(NUM_CLIENTS)

Files already downloaded and verified
Files already downloaded and verified


### Model training/evaluation

Let's continue with the usual model definition (including `set_parameters` and `get_parameters`), training and test functions:

In [3]:
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]


def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters) #state dicts contains the model layers and values
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict}) #Ordered Dict
    net.load_state_dict(state_dict, strict=True) #load model 


def train(net, trainloader, epochs: int):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(net(images), labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        #M yaha pe kaam kar raha hu 
        
            
        print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}")
        with open("file.csv",'w', newline= '') as file:
            writer = csv.writer(file)
            for epoch in range(epochs):
                writer.writerow([epoch, epoch_loss, epoch_acc])


def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

### Flower client

To implement the Flower client, we (again) create a subclass of `flwr.client.NumPyClient` and implement the three methods `get_parameters`, `fit`, and `evaluate`. Here, we also pass the `cid` to the client and use it log additional details:

In [41]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        print(f"[Client {self.cid}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=config[1])
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        print(f"Client {self.cid} loss {loss}")
        print(f"Client {self.cid} accuracy {accuracy}")
        
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}


def client_fn(cid) -> FlowerClient:
    net = Net().to(DEVICE) #Load Model from here
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    return FlowerClient(cid, net, trainloader, valloader)

Let's test what we have so far before we continue:

In [5]:
# Specify client resources if you need GPU (defaults to 1 CPU and 0 GPU)
client_resources = None
if DEVICE.type == "cuda":
    client_resources = {"num_gpus": 1}

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=5,
    config=fl.server.ServerConfig(num_rounds=10),
    client_resources=client_resources,
)

INFO flwr 2024-04-26 09:46:36,070 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
INFO flwr 2024-04-26 09:46:51,004 | app.py:180 | Flower VCE: Ray initialized with resources: {'node:127.0.0.1': 1.0, 'memory': 77794297652.0, 'object_store_memory': 37626127564.0, 'CPU': 32.0, 'GPU': 1.0}
INFO flwr 2024-04-26 09:46:51,007 | server.py:86 | Initializing global parameters
INFO flwr 2024-04-26 09:46:51,010 | server.py:273 | Requesting initial parameters from one random client
INFO flwr 2024-04-26 09:46:54,861 | server.py:277 | Received initial parameters from one random client
INFO flwr 2024-04-26 09:46:54,862 | server.py:88 | Evaluating initial parameters
INFO flwr 2024-04-26 09:46:54,863 | server.py:101 | FL starting
DEBUG flwr 2024-04-26 09:46:54,865 | server.py:218 | fit_round 1: strategy sampled 5 clients (out of 5)


 pid=16688) [Client 3] get_parameters
 pid=16688) [Client 0] fit, config: {}
 pid=16688) Epoch 1: train loss 0.06501293182373047, accuracy 0.24333333333333335
 pid=16688) Epoch 2: train loss 0.056729961186647415, accuracy 0.3382222222222222
 pid=16688) Epoch 3: train loss 0.052191298454999924, accuracy 0.4002222222222222
 pid=16688) Epoch 4: train loss 0.04877634719014168, accuracy 0.4388888888888889
 pid=16688) Epoch 5: train loss 0.04558875039219856, accuracy 0.474
 pid=16688) [Client 2] fit, config: {}
 pid=16688) Epoch 1: train loss 0.06514865905046463, accuracy 0.22466666666666665
 pid=16688) Epoch 2: train loss 0.0571310855448246, accuracy 0.32355555555555554
 pid=16688) Epoch 3: train loss 0.05288461595773697, accuracy 0.38133333333333336
 pid=16688) Epoch 4: train loss 0.049541253596544266, accuracy 0.4288888888888889
 pid=16688) Epoch 5: train loss 0.046884287148714066, accuracy 0.45466666666666666
 pid=16688) [Client 1] fit, config: {}
 pid=16688) Epoch 1: train loss 0.065227

DEBUG flwr 2024-04-26 09:48:22,472 | server.py:232 | fit_round 1 received 5 results and 0 failures
WARNING flwr 2024-04-26 09:48:22,491 | fedavg.py:243 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2024-04-26 09:48:22,492 | server.py:168 | evaluate_round 1: strategy sampled 5 clients (out of 5)


 pid=16688) Epoch 5: train loss 0.04705313593149185, accuracy 0.44555555555555554
 pid=16688) [Client 3] evaluate, config: {}
 pid=16688) Client 3 loss 0.0549422242641449
 pid=16688) Client 3 accuracy 0.42
 pid=16688) [Client 1] evaluate, config: {}
 pid=16688) Client 1 loss 0.05574341869354248
 pid=16688) Client 1 accuracy 0.394
 pid=16688) [Client 0] evaluate, config: {}
 pid=16688) Client 0 loss 0.05583895325660706
 pid=16688) Client 0 accuracy 0.378
 pid=16688) [Client 2] evaluate, config: {}
 pid=16688) Client 2 loss 0.055258041620254515
 pid=16688) Client 2 accuracy 0.406
 pid=16688) [Client 4] evaluate, config: {}


DEBUG flwr 2024-04-26 09:48:30,360 | server.py:182 | evaluate_round 1 received 5 results and 0 failures
WARNING flwr 2024-04-26 09:48:30,362 | fedavg.py:274 | No evaluate_metrics_aggregation_fn provided
DEBUG flwr 2024-04-26 09:48:30,363 | server.py:218 | fit_round 2: strategy sampled 5 clients (out of 5)


...

 pid=16688) Epoch 5: train loss 0.00449698930606246, accuracy 0.9588888888888889
 pid=16688) [Client 1] evaluate, config: {}
 pid=16688) Client 1 loss 0.063223370552063
 pid=16688) Client 1 accuracy 0.534
 pid=16688) [Client 4] evaluate, config: {}
 pid=16688) Client 4 loss 0.06229928874969482
 pid=16688) Client 4 accuracy 0.57
 pid=16688) [Client 3] evaluate, config: {}
 pid=16688) Client 3 loss 0.06798017120361328
 pid=16688) Client 3 accuracy 0.582
 pid=16688) [Client 2] evaluate, config: {}
 pid=16688) Client 2 loss 0.06594762969017029
 pid=16688) Client 2 accuracy 0.566


DEBUG flwr 2024-04-26 10:00:52,561 | server.py:182 | evaluate_round 10 received 5 results and 0 failures
INFO flwr 2024-04-26 10:00:52,562 | server.py:147 | FL finished in 837.6975861000001
INFO flwr 2024-04-26 10:00:52,564 | app.py:218 | app_fit: losses_distributed [(1, 0.055366327524185174), (2, 0.04552419316768646), (3, 0.04231918756961823), (4, 0.04187274069786072), (5, 0.04300299496650696), (6, 0.04478733510971069), (7, 0.04934098474979401), (8, 0.054115965843200685), (9, 0.05944541313648224), (10, 0.0650506513118744)]
INFO flwr 2024-04-26 10:00:52,565 | app.py:219 | app_fit: metrics_distributed_fit {}
INFO flwr 2024-04-26 10:00:52,567 | app.py:220 | app_fit: metrics_distributed {}
INFO flwr 2024-04-26 10:00:52,568 | app.py:221 | app_fit: losses_centralized []
INFO flwr 2024-04-26 10:00:52,570 | app.py:222 | app_fit: metrics_centralized {}


 pid=16688) [Client 0] evaluate, config: {}


History (loss, distributed):
	round 1: 0.055366327524185174
	round 2: 0.04552419316768646
	round 3: 0.04231918756961823
	round 4: 0.04187274069786072
	round 5: 0.04300299496650696
	round 6: 0.04478733510971069
	round 7: 0.04934098474979401
	round 8: 0.054115965843200685
	round 9: 0.05944541313648224
	round 10: 0.0650506513118744

 pid=16688) Client 0 loss 0.06580279636383056
 pid=16688) Client 0 accuracy 0.56


## Build a Strategy from scratch

Let’s overwrite the `configure_fit` method such that it passes a higher learning rate (potentially also other hyperparameters) to the optimizer of a fraction of the clients. We will keep the sampling of the clients as it is in `FedAvg` and then change the configuration dictionary (one of the `FitIns` attributes).

In [36]:
from typing import Callable, Union

from flwr.common import (
    EvaluateIns,
    EvaluateRes,
    FitIns,
    FitRes,
    MetricsAggregationFn,
    NDArrays,
    Parameters,
    Scalar,
    ndarrays_to_parameters,
    parameters_to_ndarrays,
)
from flwr.server.client_manager import ClientManager
from flwr.server.client_proxy import ClientProxy
from flwr.server.strategy.aggregate import aggregate, weighted_loss_avg


class FedCustom(fl.server.strategy.Strategy):
    def __init__(
        self,
        fraction_fit: float = 1.0,
        fraction_evaluate: float = 1.0,
        min_fit_clients: int = 2,
        min_evaluate_clients: int = 2,
        min_available_clients: int = 2,
    ) -> None:
        super().__init__()
        self.fraction_fit = fraction_fit
        self.fraction_evaluate = fraction_evaluate
        self.min_fit_clients = min_fit_clients
        self.min_evaluate_clients = min_evaluate_clients
        self.min_available_clients = min_available_clients

    def __repr__(self) -> str:
        return "FedCustom"

    def initialize_parameters(
        self, client_manager: ClientManager
    ) -> Optional[Parameters]:
        """Initialize global model parameters."""
        net = Net()
        ndarrays = get_parameters(net)
        return fl.common.ndarrays_to_parameters(ndarrays)

    def configure_fit(
        self, server_round: int, parameters: Parameters, client_manager: ClientManager
    ) -> List[Tuple[ClientProxy, FitIns]]:
        """Configure the next round of training."""

        # Sample clients
        sample_size, min_num_clients = self.num_fit_clients(
            client_manager.num_available()
        )
        clients = client_manager.sample(
            num_clients=sample_size, min_num_clients=min_num_clients
        )

        # Create custom configs
        n_clients = len(clients)
        #half_clients = n_clients // 2
        standard_config = {"lr": 0.001}
        higher_lr_config = {"lr": 0.003}
        fit_configurations = []
        for idx, client in enumerate(clients):
            fit_configurations.append((client, FitIns(parameters, standard_config)))
            # if idx < half_clients:
            #     fit_configurations.append((client, FitIns(parameters, standard_config)))
            # else:
            #     fit_configurations.append(
            #         (client, FitIns(parameters, higher_lr_config))
            #     )
        return fit_configurations

    def aggregate_fit(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, FitRes]],
        failures: List[Union[Tuple[ClientProxy, FitRes], BaseException]],
    ) -> Tuple[Optional[Parameters], Dict[str, Scalar]]:
        """Aggregate fit results using weighted average."""

        weights_results = [
            (parameters_to_ndarrays(fit_res.parameters), fit_res.num_examples)
            for _, fit_res in results
        ]
        parameters_aggregated = ndarrays_to_parameters(aggregate(weights_results))
        metrics_aggregated = {}
        return parameters_aggregated, metrics_aggregated

    def configure_evaluate(
        self, server_round: int, parameters: Parameters, client_manager: ClientManager
    ) -> List[Tuple[ClientProxy, EvaluateIns]]:
        """Configure the next round of evaluation."""
        if self.fraction_evaluate == 0.0:
            return []
        config = {}
        evaluate_ins = EvaluateIns(parameters, config)

        # Sample clients
        sample_size, min_num_clients = self.num_evaluation_clients(
            client_manager.num_available()
        )
        clients = client_manager.sample(
            num_clients=sample_size, min_num_clients=min_num_clients
        )

        # Return client/config pairs
        return [(client, evaluate_ins) for client in clients]

    def aggregate_evaluate(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, EvaluateRes]],
        failures: List[Union[Tuple[ClientProxy, EvaluateRes], BaseException]],
    ) -> Tuple[Optional[float], Dict[str, Scalar]]:
        """Aggregate evaluation losses using weighted average."""

        if not results:
            return None, {}

        loss_aggregated = weighted_loss_avg(
            [
                (evaluate_res.num_examples, evaluate_res.loss)
                for _, evaluate_res in results
            ]
        )
        metrics_aggregated = {}
        return loss_aggregated, metrics_aggregated

    def evaluate(
        self, server_round: int, parameters: Parameters
    ) -> Optional[Tuple[float, Dict[str, Scalar]]]:
        """Evaluate global model parameters using an evaluation function."""

        # Let's assume we won't perform the global model evaluation on the server side.
        return None

    def num_fit_clients(self, num_available_clients: int) -> Tuple[int, int]:
        """Return sample size and required number of clients."""
        num_clients = int(num_available_clients * self.fraction_fit)
        return max(num_clients, self.min_fit_clients), self.min_available_clients

    def num_evaluation_clients(self, num_available_clients: int) -> Tuple[int, int]:
        """Use a fraction of available clients for evaluation."""
        num_clients = int(num_available_clients * self.fraction_evaluate)
        return max(num_clients, self.min_evaluate_clients), self.min_available_clients

The only thing left is to use the newly created custom Strategy `FedCustom` when starting the experiment:

MY EXPERIMENT WITH EPOCH IN FITINS

In [43]:
from typing import Callable, Union

from flwr.common import (
    EvaluateIns,
    EvaluateRes,
    FitIns,
    FitRes,
    MetricsAggregationFn,
    NDArrays,
    Parameters,
    Scalar,
    ndarrays_to_parameters,
    parameters_to_ndarrays,
)
from flwr.server.client_manager import ClientManager
from flwr.server.client_proxy import ClientProxy
from flwr.server.strategy.aggregate import aggregate, weighted_loss_avg


class FedCustom(fl.server.strategy.Strategy):
    def __init__(
        self,
        fraction_fit: float = 1.0,
        fraction_evaluate: float = 1.0,
        min_fit_clients: int = 2,
        min_evaluate_clients: int = 2,
        min_available_clients: int = 2,
    ) -> None:
        super().__init__()
        self.fraction_fit = fraction_fit
        self.fraction_evaluate = fraction_evaluate
        self.min_fit_clients = min_fit_clients
        self.min_evaluate_clients = min_evaluate_clients
        self.min_available_clients = min_available_clients

    def __repr__(self) -> str:
        return "FedCustom"

    def initialize_parameters(
        self, client_manager: ClientManager
    ) -> Optional[Parameters]:
        """Initialize global model parameters."""
        net = Net()
        ndarrays = get_parameters(net)
        return fl.common.ndarrays_to_parameters(ndarrays)

    def configure_fit(
        self, server_round: int, parameters: Parameters, client_manager: ClientManager
    ) -> List[Tuple[ClientProxy, FitIns]]:
        """Configure the next round of training."""

        # Sample clients
        sample_size, min_num_clients = self.num_fit_clients(
            client_manager.num_available()
        )
        clients = client_manager.sample(
            num_clients=sample_size, min_num_clients=min_num_clients
        )

        # Create custom configs
        n_clients = len(clients)
        #half_clients = n_clients // 2
        standard_config = {"lr": 0.001, "epoch": 2}
        epooc = {"epoch": 2}
        higher_lr_config = {"lr": 0.003}
        fit_configurations = []
        for idx, client in enumerate(clients):
            fit_configurations.append((client, FitIns(parameters, standard_config)))
            # if idx < half_clients:
            #     fit_configurations.append((client, FitIns(parameters, standard_config)))
            # else:
            #     fit_configurations.append(
            #         (client, FitIns(parameters, higher_lr_config))
            #     )
        return fit_configurations

    def aggregate_fit(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, FitRes]],
        failures: List[Union[Tuple[ClientProxy, FitRes], BaseException]],
    ) -> Tuple[Optional[Parameters], Dict[str, Scalar]]:
        """Aggregate fit results using weighted average."""

        weights_results = [
            (parameters_to_ndarrays(fit_res.parameters), fit_res.num_examples)
            for _, fit_res in results
        ]
        parameters_aggregated = ndarrays_to_parameters(aggregate(weights_results))
        metrics_aggregated = {}
        return parameters_aggregated, metrics_aggregated

    def configure_evaluate(
        self, server_round: int, parameters: Parameters, client_manager: ClientManager
    ) -> List[Tuple[ClientProxy, EvaluateIns]]:
        """Configure the next round of evaluation."""
        if self.fraction_evaluate == 0.0:
            return []
        config = {"epoch": str(2)}
        evaluate_ins = EvaluateIns(parameters, config)

        # Sample clients
        sample_size, min_num_clients = self.num_evaluation_clients(
            client_manager.num_available()
        )
        clients = client_manager.sample(
            num_clients=sample_size, min_num_clients=min_num_clients
        )

        # Return client/config pairs
        return [(client, evaluate_ins) for client in clients]

    def aggregate_evaluate(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, EvaluateRes]],
        failures: List[Union[Tuple[ClientProxy, EvaluateRes], BaseException]],
    ) -> Tuple[Optional[float], Dict[str, Scalar]]:
        """Aggregate evaluation losses using weighted average."""

        if not results:
            return None, {}

        loss_aggregated = weighted_loss_avg(
            [
                (evaluate_res.num_examples, evaluate_res.loss)
                for _, evaluate_res in results
            ]
        )
        metrics_aggregated = {}
        return loss_aggregated, metrics_aggregated

    def evaluate(
        self, server_round: int, parameters: Parameters
    ) -> Optional[Tuple[float, Dict[str, Scalar]]]:
        """Evaluate global model parameters using an evaluation function."""

        # Let's assume we won't perform the global model evaluation on the server side.
        return None

    def num_fit_clients(self, num_available_clients: int) -> Tuple[int, int]:
        """Return sample size and required number of clients."""
        num_clients = int(num_available_clients * self.fraction_fit)
        return max(num_clients, self.min_fit_clients), self.min_available_clients

    def num_evaluation_clients(self, num_available_clients: int) -> Tuple[int, int]:
        """Use a fraction of available clients for evaluation."""
        num_clients = int(num_available_clients * self.fraction_evaluate)
        return max(num_clients, self.min_evaluate_clients), self.min_available_clients

In [44]:
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=5,
    config=fl.server.ServerConfig(num_rounds=10),
    strategy=FedCustom(),  # <-- pass the new strategy here
    client_resources=client_resources,
)

INFO flwr 2024-04-27 10:27:37,258 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)


INFO flwr 2024-04-27 10:27:52,243 | app.py:180 | Flower VCE: Ray initialized with resources: {'node:127.0.0.1': 1.0, 'object_store_memory': 29394678988.0, 'memory': 58789357979.0, 'CPU': 32.0, 'GPU': 1.0}
INFO flwr 2024-04-27 10:27:52,245 | server.py:86 | Initializing global parameters
INFO flwr 2024-04-27 10:27:52,250 | server.py:269 | Using initial parameters provided by strategy
INFO flwr 2024-04-27 10:27:52,250 | server.py:88 | Evaluating initial parameters
INFO flwr 2024-04-27 10:27:52,252 | server.py:101 | FL starting
DEBUG flwr 2024-04-27 10:27:52,252 | server.py:218 | fit_round 1: strategy sampled 5 clients (out of 5)
ERROR flwr 2024-04-27 10:28:01,599 | ray_client_proxy.py:87 | ray::launch_and_fit() (pid=28080, ip=127.0.0.1)
  File "python\ray\_raylet.pyx", line 640, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 644, in ray._raylet.execute_task
  File "c:\Users\Admin\anaconda3\envs\flwrpytorch\lib\site-packages\flwr\simulation\ray_transport\ray_client_proxy

 pid=28080) [Client 2] fit, config: {'lr': 0.001, 'epoch': 2}
 pid=23036) [Client 1] fit, config: {'lr': 0.001, 'epoch': 2}
 pid=24556) [Client 3] fit, config: {'lr': 0.001, 'epoch': 2}


ERROR flwr 2024-04-27 10:28:01,886 | ray_client_proxy.py:87 | ray::launch_and_fit() (pid=29316, ip=127.0.0.1)
  File "python\ray\_raylet.pyx", line 640, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 644, in ray._raylet.execute_task
  File "c:\Users\Admin\anaconda3\envs\flwrpytorch\lib\site-packages\flwr\simulation\ray_transport\ray_client_proxy.py", line 148, in launch_and_fit
    return maybe_call_fit(
  File "c:\Users\Admin\anaconda3\envs\flwrpytorch\lib\site-packages\flwr\client\client.py", line 184, in maybe_call_fit
    return client.fit(fit_ins)
  File "c:\Users\Admin\anaconda3\envs\flwrpytorch\lib\site-packages\flwr\client\app.py", line 297, in _fit
    results = self.numpy_client.fit(parameters, ins.config)  # type: ignore
  File "C:\Users\Admin\AppData\Local\Temp\ipykernel_11608\511400589.py", line 15, in fit
KeyError: 1
ERROR flwr 2024-04-27 10:28:01,937 | ray_client_proxy.py:87 | ray::launch_and_fit() (pid=20092, ip=127.0.0.1)
  File "python\ray\_raylet.p

 pid=29316) [Client 4] fit, config: {'lr': 0.001, 'epoch': 2}
 pid=20092) [Client 0] fit, config: {'lr': 0.001, 'epoch': 2}


...

ERROR flwr 2024-04-27 10:28:15,613 | ray_client_proxy.py:87 | ray::launch_and_fit() (pid=24556, ip=127.0.0.1)
  File "python\ray\_raylet.pyx", line 640, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 644, in ray._raylet.execute_task
  File "c:\Users\Admin\anaconda3\envs\flwrpytorch\lib\site-packages\flwr\simulation\ray_transport\ray_client_proxy.py", line 148, in launch_and_fit
    return maybe_call_fit(
  File "c:\Users\Admin\anaconda3\envs\flwrpytorch\lib\site-packages\flwr\client\client.py", line 184, in maybe_call_fit
    return client.fit(fit_ins)
  File "c:\Users\Admin\anaconda3\envs\flwrpytorch\lib\site-packages\flwr\client\app.py", line 297, in _fit
    results = self.numpy_client.fit(parameters, ins.config)  # type: ignore
  File "C:\Users\Admin\AppData\Local\Temp\ipykernel_11608\511400589.py", line 14, in fit
  File "C:\Users\Admin\AppData\Local\Temp\ipykernel_11608\1504573457.py", line 28, in set_parameters
  File "c:\Users\Admin\anaconda3\envs\flwrpytorch

 pid=24556) [Client 2] fit, config: {'lr': 0.001, 'epoch': 2}
 pid=24556) [Client 0] fit, config: {'lr': 0.001, 'epoch': 2}
 pid=24556) [Client 1] fit, config: {'lr': 0.001, 'epoch': 2}


ERROR flwr 2024-04-27 10:28:22,867 | ray_client_proxy.py:104 | ray::launch_and_evaluate() (pid=24556, ip=127.0.0.1)
  File "python\ray\_raylet.pyx", line 640, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 644, in ray._raylet.execute_task
  File "c:\Users\Admin\anaconda3\envs\flwrpytorch\lib\site-packages\flwr\simulation\ray_transport\ray_client_proxy.py", line 160, in launch_and_evaluate
    return maybe_call_evaluate(
  File "c:\Users\Admin\anaconda3\envs\flwrpytorch\lib\site-packages\flwr\client\client.py", line 205, in maybe_call_evaluate
    return client.evaluate(evaluate_ins)
  File "c:\Users\Admin\anaconda3\envs\flwrpytorch\lib\site-packages\flwr\client\app.py", line 321, in _evaluate
    results = self.numpy_client.evaluate(parameters, ins.config)  # type: ignore
  File "C:\Users\Admin\AppData\Local\Temp\ipykernel_11608\511400589.py", line 20, in evaluate
  File "C:\Users\Admin\AppData\Local\Temp\ipykernel_11608\1504573457.py", line 28, in set_parameters
  F

 pid=20092) [Client 4] fit, config: {'lr': 0.001, 'epoch': 2}
 pid=24556) [Client 3] evaluate, config: {'epoch': '2'}
 pid=24556) [Client 4] evaluate, config: {'epoch': '2'}
 pid=24556) [Client 1] evaluate, config: {'epoch': '2'}
 pid=29316) [Client 0] evaluate, config: {'epoch': '2'}
 pid=20092) [Client 2] evaluate, config: {'epoch': '2'}


KeyboardInterrupt: 

In [7]:
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=5,
    config=fl.server.ServerConfig(num_rounds=10),
    strategy=FedCustom(),  # <-- pass the new strategy here
    client_resources=client_resources,
)

INFO flwr 2024-04-26 10:12:05,143 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)


INFO flwr 2024-04-26 10:12:20,952 | app.py:180 | Flower VCE: Ray initialized with resources: {'GPU': 1.0, 'CPU': 32.0, 'node:127.0.0.1': 1.0, 'object_store_memory': 35730602803.0, 'memory': 73371406541.0}
INFO flwr 2024-04-26 10:12:20,955 | server.py:86 | Initializing global parameters
INFO flwr 2024-04-26 10:12:20,967 | server.py:269 | Using initial parameters provided by strategy
INFO flwr 2024-04-26 10:12:20,968 | server.py:88 | Evaluating initial parameters
INFO flwr 2024-04-26 10:12:20,970 | server.py:101 | FL starting
DEBUG flwr 2024-04-26 10:12:20,971 | server.py:218 | fit_round 1: strategy sampled 5 clients (out of 5)


 pid=25688) [Client 0] fit, config: {'lr': 0.001}
 pid=25688) Epoch 1: train loss 0.06557507067918777, accuracy 0.22866666666666666
 pid=25688) Epoch 2: train loss 0.05846356227993965, accuracy 0.3217777777777778
 pid=25688) Epoch 3: train loss 0.05338079109787941, accuracy 0.37444444444444447
 pid=25688) Epoch 4: train loss 0.05022795870900154, accuracy 0.4142222222222222
 pid=25688) Epoch 5: train loss 0.04769791290163994, accuracy 0.4568888888888889
 pid=25688) [Client 3] fit, config: {'lr': 0.001}
 pid=25688) Epoch 1: train loss 0.06624044477939606, accuracy 0.21355555555555555
 pid=25688) Epoch 2: train loss 0.05896100029349327, accuracy 0.3028888888888889
 pid=25688) Epoch 3: train loss 0.054232947528362274, accuracy 0.37044444444444447
 pid=25688) Epoch 4: train loss 0.05087036266922951, accuracy 0.4042222222222222
 pid=25688) Epoch 5: train loss 0.04829408600926399, accuracy 0.43866666666666665
 pid=25688) [Client 4] fit, config: {'lr': 0.001}
 pid=25688) Epoch 1: train loss 0.

DEBUG flwr 2024-04-26 10:13:46,092 | server.py:232 | fit_round 1 received 5 results and 0 failures
DEBUG flwr 2024-04-26 10:13:46,111 | server.py:168 | evaluate_round 1: strategy sampled 5 clients (out of 5)


 pid=25688) Epoch 5: train loss 0.04848460108041763, accuracy 0.43177777777777776
 pid=25688) [Client 1] evaluate, config: {}
 pid=25688) Client 1 loss 0.05722716760635376
 pid=25688) Client 1 accuracy 0.376
 pid=25688) [Client 0] evaluate, config: {}
 pid=25688) Client 0 loss 0.057526315212249754
 pid=25688) Client 0 accuracy 0.404
 pid=25688) [Client 2] evaluate, config: {}
 pid=25688) Client 2 loss 0.05748161768913269
 pid=25688) Client 2 accuracy 0.392
 pid=25688) [Client 3] evaluate, config: {}
 pid=25688) Client 3 loss 0.056574954986572265
 pid=25688) Client 3 accuracy 0.376


...

 pid=25688) Epoch 5: train loss 0.0065334029495716095, accuracy 0.9375555555555556
 pid=25688) [Client 0] evaluate, config: {}
 pid=25688) Client 0 loss 0.0655604281425476
 pid=25688) Client 0 accuracy 0.558
 pid=25688) [Client 3] evaluate, config: {}
 pid=25688) Client 3 loss 0.06983086109161377
 pid=25688) Client 3 accuracy 0.524
 pid=25688) [Client 1] evaluate, config: {}
 pid=25688) Client 1 loss 0.059871720433235166
 pid=25688) Client 1 accuracy 0.56
 pid=25688) [Client 4] evaluate, config: {}


DEBUG flwr 2024-04-26 10:26:11,897 | server.py:182 | evaluate_round 10 received 5 results and 0 failures


 pid=25688) Client 4 loss 0.057930799007415774
 pid=25688) Client 4 accuracy 0.562
 pid=25688) [Client 2] evaluate, config: {}


INFO flwr 2024-04-26 10:26:11,899 | server.py:147 | FL finished in 830.9280965
INFO flwr 2024-04-26 10:26:11,901 | app.py:218 | app_fit: losses_distributed [(1, 0.057120275640487675), (2, 0.04623582530021668), (3, 0.04284876246452331), (4, 0.04314018495082855), (5, 0.04397216129302978), (6, 0.046886711287498474), (7, 0.04969739663600921), (8, 0.05484497110843659), (9, 0.05937500901222229), (10, 0.06456811034679413)]
INFO flwr 2024-04-26 10:26:11,902 | app.py:219 | app_fit: metrics_distributed_fit {}
INFO flwr 2024-04-26 10:26:11,904 | app.py:220 | app_fit: metrics_distributed {}
INFO flwr 2024-04-26 10:26:11,906 | app.py:221 | app_fit: losses_centralized []
INFO flwr 2024-04-26 10:26:11,907 | app.py:222 | app_fit: metrics_centralized {}


History (loss, distributed):
	round 1: 0.057120275640487675
	round 2: 0.04623582530021668
	round 3: 0.04284876246452331
	round 4: 0.04314018495082855
	round 5: 0.04397216129302978
	round 6: 0.046886711287498474
	round 7: 0.04969739663600921
	round 8: 0.05484497110843659
	round 9: 0.05937500901222229
	round 10: 0.06456811034679413

In [8]:
from typing import Callable, Union

from flwr.common import (
    EvaluateIns,
    EvaluateRes,
    FitIns,
    FitRes,
    MetricsAggregationFn,
    NDArrays,
    Parameters,
    Scalar,
    ndarrays_to_parameters,
    parameters_to_ndarrays,
)
from flwr.server.client_manager import ClientManager
from flwr.server.client_proxy import ClientProxy
from flwr.server.strategy.aggregate import aggregate, weighted_loss_avg


class FedCustom(fl.server.strategy.Strategy):
    def __init__(
        self,
        fraction_fit: float = 1.0,
        fraction_evaluate: float = 1.0,
        min_fit_clients: int = 2,
        min_evaluate_clients: int = 2,
        min_available_clients: int = 2,
    ) -> None:
        super().__init__()
        self.fraction_fit = fraction_fit
        self.fraction_evaluate = fraction_evaluate
        self.min_fit_clients = min_fit_clients
        self.min_evaluate_clients = min_evaluate_clients
        self.min_available_clients = min_available_clients

    def __repr__(self) -> str:
        return "FedCustom"

    def initialize_parameters(
        self, client_manager: ClientManager
    ) -> Optional[Parameters]:
        """Initialize global model parameters."""
        net = Net()
        ndarrays = get_parameters(net)
        return fl.common.ndarrays_to_parameters(ndarrays)

    def configure_fit(
        self, server_round: int, parameters: Parameters, client_manager: ClientManager
    ) -> List[Tuple[ClientProxy, FitIns]]:
        """Configure the next round of training."""

        # Sample clients
        sample_size, min_num_clients = self.num_fit_clients(
            client_manager.num_available()
        )
        clients = client_manager.sample(
            num_clients=sample_size, min_num_clients=min_num_clients
        )

        # Create custom configs
        n_clients = len(clients)
        #half_clients = n_clients // 2
        standard_config = {"lr": 0.0001} #More Smaller LR
        higher_lr_config = {"lr": 0.003}
        fit_configurations = []
        for idx, client in enumerate(clients):
            fit_configurations.append((client, FitIns(parameters, standard_config)))
            # if idx < half_clients:
            #     fit_configurations.append((client, FitIns(parameters, standard_config)))
            # else:
            #     fit_configurations.append(
            #         (client, FitIns(parameters, higher_lr_config))
            #     )
        return fit_configurations

    def aggregate_fit(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, FitRes]],
        failures: List[Union[Tuple[ClientProxy, FitRes], BaseException]],
    ) -> Tuple[Optional[Parameters], Dict[str, Scalar]]:
        """Aggregate fit results using weighted average."""

        weights_results = [
            (parameters_to_ndarrays(fit_res.parameters), fit_res.num_examples)
            for _, fit_res in results
        ]
        parameters_aggregated = ndarrays_to_parameters(aggregate(weights_results))
        metrics_aggregated = {}
        return parameters_aggregated, metrics_aggregated

    def configure_evaluate(
        self, server_round: int, parameters: Parameters, client_manager: ClientManager
    ) -> List[Tuple[ClientProxy, EvaluateIns]]:
        """Configure the next round of evaluation."""
        if self.fraction_evaluate == 0.0:
            return []
        config = {}
        evaluate_ins = EvaluateIns(parameters, config)

        # Sample clients
        sample_size, min_num_clients = self.num_evaluation_clients(
            client_manager.num_available()
        )
        clients = client_manager.sample(
            num_clients=sample_size, min_num_clients=min_num_clients
        )

        # Return client/config pairs
        return [(client, evaluate_ins) for client in clients]

    def aggregate_evaluate(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, EvaluateRes]],
        failures: List[Union[Tuple[ClientProxy, EvaluateRes], BaseException]],
    ) -> Tuple[Optional[float], Dict[str, Scalar]]:
        """Aggregate evaluation losses using weighted average."""

        if not results:
            return None, {}

        loss_aggregated = weighted_loss_avg(
            [
                (evaluate_res.num_examples, evaluate_res.loss)
                for _, evaluate_res in results
            ]
        )
        metrics_aggregated = {}
        return loss_aggregated, metrics_aggregated

    def evaluate(
        self, server_round: int, parameters: Parameters
    ) -> Optional[Tuple[float, Dict[str, Scalar]]]:
        """Evaluate global model parameters using an evaluation function."""

        # Let's assume we won't perform the global model evaluation on the server side.
        return None

    def num_fit_clients(self, num_available_clients: int) -> Tuple[int, int]:
        """Return sample size and required number of clients."""
        num_clients = int(num_available_clients * self.fraction_fit)
        return max(num_clients, self.min_fit_clients), self.min_available_clients

    def num_evaluation_clients(self, num_available_clients: int) -> Tuple[int, int]:
        """Use a fraction of available clients for evaluation."""
        num_clients = int(num_available_clients * self.fraction_evaluate)
        return max(num_clients, self.min_evaluate_clients), self.min_available_clients

 pid=25688) Client 2 loss 0.06964674305915833
 pid=25688) Client 2 accuracy 0.524


In [9]:
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=5,
    config=fl.server.ServerConfig(num_rounds=10),
    strategy=FedCustom(),  # <-- pass the new strategy here
    client_resources=client_resources,
)

INFO flwr 2024-04-26 10:26:12,056 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
INFO flwr 2024-04-26 10:26:26,708 | app.py:180 | Flower VCE: Ray initialized with resources: {'CPU': 32.0, 'node:127.0.0.1': 1.0, 'GPU': 1.0, 'object_store_memory': 34135699046.0, 'memory': 69649964442.0}
INFO flwr 2024-04-26 10:26:26,712 | server.py:86 | Initializing global parameters
INFO flwr 2024-04-26 10:26:26,723 | server.py:269 | Using initial parameters provided by strategy
INFO flwr 2024-04-26 10:26:26,725 | server.py:88 | Evaluating initial parameters
INFO flwr 2024-04-26 10:26:26,727 | server.py:101 | FL starting
DEBUG flwr 2024-04-26 10:26:26,730 | server.py:218 | fit_round 1: strategy sampled 5 clients (out of 5)


 pid=3148) [Client 3] fit, config: {'lr': 0.0001}
 pid=3148) Epoch 1: train loss 0.0646684318780899, accuracy 0.22644444444444445
 pid=3148) Epoch 2: train loss 0.056407880038022995, accuracy 0.32866666666666666
 pid=3148) Epoch 3: train loss 0.0523478202521801, accuracy 0.3848888888888889
 pid=3148) Epoch 4: train loss 0.049011457711458206, accuracy 0.42644444444444446
 pid=3148) Epoch 5: train loss 0.04737198352813721, accuracy 0.444
 pid=3148) [Client 2] fit, config: {'lr': 0.0001}
 pid=3148) Epoch 1: train loss 0.06500020623207092, accuracy 0.22555555555555556
 pid=3148) Epoch 2: train loss 0.0567382387816906, accuracy 0.3313333333333333
 pid=3148) Epoch 3: train loss 0.05330170691013336, accuracy 0.3695555555555556
 pid=3148) Epoch 4: train loss 0.05091230943799019, accuracy 0.39711111111111114
 pid=3148) Epoch 5: train loss 0.049259793013334274, accuracy 0.42244444444444446
 pid=3148) [Client 1] fit, config: {'lr': 0.0001}
 pid=3148) Epoch 1: train loss 0.0641135647892952, accura

DEBUG flwr 2024-04-26 10:27:51,896 | server.py:232 | fit_round 1 received 5 results and 0 failures
DEBUG flwr 2024-04-26 10:27:51,915 | server.py:168 | evaluate_round 1: strategy sampled 5 clients (out of 5)


 pid=3148) Epoch 5: train loss 0.046836476773023605, accuracy 0.45644444444444443
 pid=3148) [Client 1] evaluate, config: {}
 pid=3148) Client 1 loss 0.05805980658531189
 pid=3148) Client 1 accuracy 0.376
 pid=3148) [Client 0] evaluate, config: {}
 pid=3148) Client 0 loss 0.05745409917831421
 pid=3148) Client 0 accuracy 0.394
 pid=3148) [Client 4] evaluate, config: {}
 pid=3148) Client 4 loss 0.05654742360115051
 pid=3148) Client 4 accuracy 0.374
 pid=3148) [Client 3] evaluate, config: {}


DEBUG flwr 2024-04-26 10:28:00,044 | server.py:182 | evaluate_round 1 received 5 results and 0 failures
DEBUG flwr 2024-04-26 10:28:00,046 | server.py:218 | fit_round 2: strategy sampled 5 clients (out of 5)


...

DEBUG flwr 2024-04-26 10:40:05,354 | server.py:232 | fit_round 10 received 5 results and 0 failures
DEBUG flwr 2024-04-26 10:40:05,378 | server.py:168 | evaluate_round 10: strategy sampled 5 clients (out of 5)


 pid=3148) Epoch 5: train loss 0.00501002324745059, accuracy 0.9522222222222222
 pid=3148) [Client 2] evaluate, config: {}
 pid=3148) Client 2 loss 0.06329944205284119
 pid=3148) Client 2 accuracy 0.568
 pid=3148) [Client 4] evaluate, config: {}
 pid=3148) Client 4 loss 0.06267336559295654
 pid=3148) Client 4 accuracy 0.534
 pid=3148) [Client 1] evaluate, config: {}
 pid=3148) Client 1 loss 0.062106043219566344
 pid=3148) Client 1 accuracy 0.576
 pid=3148) [Client 0] evaluate, config: {}
 pid=3148) Client 0 loss 0.06362659239768982
 pid=3148) Client 0 accuracy 0.548


DEBUG flwr 2024-04-26 10:40:12,763 | server.py:182 | evaluate_round 10 received 5 results and 0 failures
INFO flwr 2024-04-26 10:40:12,765 | server.py:147 | FL finished in 826.0354007999999
INFO flwr 2024-04-26 10:40:12,767 | app.py:218 | app_fit: losses_distributed [(1, 0.057307543420791626), (2, 0.045569618129730226), (3, 0.042455916547775266), (4, 0.041634908509254456), (5, 0.04203821234703063), (6, 0.04487658135890961), (7, 0.0477320716381073), (8, 0.053122343039512646), (9, 0.05906837012767792), (10, 0.06344441199302672)]
INFO flwr 2024-04-26 10:40:12,768 | app.py:219 | app_fit: metrics_distributed_fit {}
INFO flwr 2024-04-26 10:40:12,769 | app.py:220 | app_fit: metrics_distributed {}
INFO flwr 2024-04-26 10:40:12,771 | app.py:221 | app_fit: losses_centralized []
INFO flwr 2024-04-26 10:40:12,772 | app.py:222 | app_fit: metrics_centralized {}


 pid=3148) [Client 3] evaluate, config: {}


History (loss, distributed):
	round 1: 0.057307543420791626
	round 2: 0.045569618129730226
	round 3: 0.042455916547775266
	round 4: 0.041634908509254456
	round 5: 0.04203821234703063
	round 6: 0.04487658135890961
	round 7: 0.0477320716381073
	round 8: 0.053122343039512646
	round 9: 0.05906837012767792
	round 10: 0.06344441199302672

In [10]:
from typing import Callable, Union

from flwr.common import (
    EvaluateIns,
    EvaluateRes,
    FitIns,
    FitRes,
    MetricsAggregationFn,
    NDArrays,
    Parameters,
    Scalar,
    ndarrays_to_parameters,
    parameters_to_ndarrays,
)
from flwr.server.client_manager import ClientManager
from flwr.server.client_proxy import ClientProxy
from flwr.server.strategy.aggregate import aggregate, weighted_loss_avg


class FedCustom(fl.server.strategy.Strategy):
    def __init__(
        self,
        fraction_fit: float = 1.0,
        fraction_evaluate: float = 1.0,
        min_fit_clients: int = 2,
        min_evaluate_clients: int = 2,
        min_available_clients: int = 2,
    ) -> None:
        super().__init__()
        self.fraction_fit = fraction_fit
        self.fraction_evaluate = fraction_evaluate
        self.min_fit_clients = min_fit_clients
        self.min_evaluate_clients = min_evaluate_clients
        self.min_available_clients = min_available_clients

    def __repr__(self) -> str:
        return "FedCustom"

    def initialize_parameters(
        self, client_manager: ClientManager
    ) -> Optional[Parameters]:
        """Initialize global model parameters."""
        net = Net()
        ndarrays = get_parameters(net)
        return fl.common.ndarrays_to_parameters(ndarrays)

    def configure_fit(
        self, server_round: int, parameters: Parameters, client_manager: ClientManager
    ) -> List[Tuple[ClientProxy, FitIns]]:
        """Configure the next round of training."""

        # Sample clients
        sample_size, min_num_clients = self.num_fit_clients(
            client_manager.num_available()
        )
        clients = client_manager.sample(
            num_clients=sample_size, min_num_clients=min_num_clients
        )

        # Create custom configs
        n_clients = len(clients)
        #half_clients = n_clients // 2
        standard_config = {"lr": 0.01} #More Tighter LR
        higher_lr_config = {"lr": 0.003}
        fit_configurations = []
        for idx, client in enumerate(clients):
            fit_configurations.append((client, FitIns(parameters, standard_config)))
            # if idx < half_clients:
            #     fit_configurations.append((client, FitIns(parameters, standard_config)))
            # else:
            #     fit_configurations.append(
            #         (client, FitIns(parameters, higher_lr_config))
            #     )
        return fit_configurations

    def aggregate_fit(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, FitRes]],
        failures: List[Union[Tuple[ClientProxy, FitRes], BaseException]],
    ) -> Tuple[Optional[Parameters], Dict[str, Scalar]]:
        """Aggregate fit results using weighted average."""

        weights_results = [
            (parameters_to_ndarrays(fit_res.parameters), fit_res.num_examples)
            for _, fit_res in results
        ]
        parameters_aggregated = ndarrays_to_parameters(aggregate(weights_results))
        metrics_aggregated = {}
        return parameters_aggregated, metrics_aggregated

    def configure_evaluate(
        self, server_round: int, parameters: Parameters, client_manager: ClientManager
    ) -> List[Tuple[ClientProxy, EvaluateIns]]:
        """Configure the next round of evaluation."""
        if self.fraction_evaluate == 0.0:
            return []
        config = {}
        evaluate_ins = EvaluateIns(parameters, config)

        # Sample clients
        sample_size, min_num_clients = self.num_evaluation_clients(
            client_manager.num_available()
        )
        clients = client_manager.sample(
            num_clients=sample_size, min_num_clients=min_num_clients
        )

        # Return client/config pairs
        return [(client, evaluate_ins) for client in clients]

    def aggregate_evaluate(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, EvaluateRes]],
        failures: List[Union[Tuple[ClientProxy, EvaluateRes], BaseException]],
    ) -> Tuple[Optional[float], Dict[str, Scalar]]:
        """Aggregate evaluation losses using weighted average."""

        if not results:
            return None, {}

        loss_aggregated = weighted_loss_avg(
            [
                (evaluate_res.num_examples, evaluate_res.loss)
                for _, evaluate_res in results
            ]
        )
        metrics_aggregated = {}
        return loss_aggregated, metrics_aggregated

    def evaluate(
        self, server_round: int, parameters: Parameters
    ) -> Optional[Tuple[float, Dict[str, Scalar]]]:
        """Evaluate global model parameters using an evaluation function."""

        # Let's assume we won't perform the global model evaluation on the server side.
        return None

    def num_fit_clients(self, num_available_clients: int) -> Tuple[int, int]:
        """Return sample size and required number of clients."""
        num_clients = int(num_available_clients * self.fraction_fit)
        return max(num_clients, self.min_fit_clients), self.min_available_clients

    def num_evaluation_clients(self, num_available_clients: int) -> Tuple[int, int]:
        """Use a fraction of available clients for evaluation."""
        num_clients = int(num_available_clients * self.fraction_evaluate)
        return max(num_clients, self.min_evaluate_clients), self.min_available_clients

 pid=3148) Client 3 loss 0.06551661670207977
 pid=3148) Client 3 accuracy 0.544


In [11]:
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=5,
    config=fl.server.ServerConfig(num_rounds=10),
    strategy=FedCustom(),  # <-- pass the new strategy here
    client_resources=client_resources,
)

INFO flwr 2024-04-26 10:40:12,888 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
INFO flwr 2024-04-26 10:40:27,605 | app.py:180 | Flower VCE: Ray initialized with resources: {'GPU': 1.0, 'node:127.0.0.1': 1.0, 'object_store_memory': 34110701568.0, 'memory': 69591636992.0, 'CPU': 32.0}
INFO flwr 2024-04-26 10:40:27,607 | server.py:86 | Initializing global parameters
INFO flwr 2024-04-26 10:40:27,619 | server.py:269 | Using initial parameters provided by strategy
INFO flwr 2024-04-26 10:40:27,620 | server.py:88 | Evaluating initial parameters
INFO flwr 2024-04-26 10:40:27,622 | server.py:101 | FL starting
DEBUG flwr 2024-04-26 10:40:27,624 | server.py:218 | fit_round 1: strategy sampled 5 clients (out of 5)


 pid=8992) [Client 0] fit, config: {'lr': 0.01}
 pid=8992) Epoch 1: train loss 0.06484240293502808, accuracy 0.23622222222222222
 pid=8992) Epoch 2: train loss 0.0570756234228611, accuracy 0.33644444444444443
 pid=8992) Epoch 3: train loss 0.051969073712825775, accuracy 0.3948888888888889
 pid=8992) Epoch 4: train loss 0.048612650483846664, accuracy 0.43977777777777777
 pid=8992) Epoch 5: train loss 0.04648016765713692, accuracy 0.4597777777777778
 pid=8992) [Client 4] fit, config: {'lr': 0.01}
 pid=8992) Epoch 1: train loss 0.06496812403202057, accuracy 0.22333333333333333
 pid=8992) Epoch 2: train loss 0.05662725493311882, accuracy 0.3377777777777778
 pid=8992) Epoch 3: train loss 0.05184600502252579, accuracy 0.39444444444444443
 pid=8992) Epoch 4: train loss 0.04871721565723419, accuracy 0.43777777777777777
 pid=8992) Epoch 5: train loss 0.04622182250022888, accuracy 0.46755555555555556
 pid=8992) [Client 2] fit, config: {'lr': 0.01}
 pid=8992) Epoch 1: train loss 0.064651764929294

DEBUG flwr 2024-04-26 10:41:52,592 | server.py:232 | fit_round 1 received 5 results and 0 failures
DEBUG flwr 2024-04-26 10:41:52,610 | server.py:168 | evaluate_round 1: strategy sampled 5 clients (out of 5)


 pid=8992) Epoch 5: train loss 0.04640515521168709, accuracy 0.45644444444444443
 pid=8992) [Client 2] evaluate, config: {}
 pid=8992) Client 2 loss 0.05577287483215332
 pid=8992) Client 2 accuracy 0.388
 pid=8992) [Client 3] evaluate, config: {}
 pid=8992) Client 3 loss 0.0561391761302948
 pid=8992) Client 3 accuracy 0.348
 pid=8992) [Client 1] evaluate, config: {}
 pid=8992) Client 1 loss 0.055274890661239626
 pid=8992) Client 1 accuracy 0.394
 pid=8992) [Client 4] evaluate, config: {}
 pid=8992) Client 4 loss 0.05580915641784668
 pid=8992) Client 4 accuracy 0.358
 pid=8992) [Client 0] evaluate, config: {}


DEBUG flwr 2024-04-26 10:42:00,271 | server.py:182 | evaluate_round 1 received 5 results and 0 failures
DEBUG flwr 2024-04-26 10:42:00,273 | server.py:218 | fit_round 2: strategy sampled 5 clients (out of 5)


...

 pid=8992) Epoch 5: train loss 0.005459274165332317, accuracy 0.9491111111111111
 pid=8992) [Client 0] evaluate, config: {}
 pid=8992) Client 0 loss 0.06394584035873413
 pid=8992) Client 0 accuracy 0.562
 pid=8992) [Client 2] evaluate, config: {}
 pid=8992) Client 2 loss 0.0696816189289093
 pid=8992) Client 2 accuracy 0.512
 pid=8992) [Client 4] evaluate, config: {}
 pid=8992) Client 4 loss 0.0615974051952362
 pid=8992) Client 4 accuracy 0.524
 pid=8992) [Client 1] evaluate, config: {}


DEBUG flwr 2024-04-26 10:54:23,402 | server.py:182 | evaluate_round 10 received 5 results and 0 failures
INFO flwr 2024-04-26 10:54:23,404 | server.py:147 | FL finished in 835.7796104999998
INFO flwr 2024-04-26 10:54:23,406 | app.py:218 | app_fit: losses_distributed [(1, 0.05587423934936524), (2, 0.044970293760299686), (3, 0.04253035924434662), (4, 0.042628040552139285), (5, 0.044089994812011715), (6, 0.0470378776550293), (7, 0.05151897335052491), (8, 0.05631762299537659), (9, 0.062256294393539434), (10, 0.06569828534126282)]
INFO flwr 2024-04-26 10:54:23,407 | app.py:219 | app_fit: metrics_distributed_fit {}
INFO flwr 2024-04-26 10:54:23,409 | app.py:220 | app_fit: metrics_distributed {}
INFO flwr 2024-04-26 10:54:23,410 | app.py:221 | app_fit: losses_centralized []
INFO flwr 2024-04-26 10:54:23,412 | app.py:222 | app_fit: metrics_centralized {}


 pid=8992) Client 1 loss 0.0650769727230072
 pid=8992) Client 1 accuracy 0.524
 pid=8992) [Client 3] evaluate, config: {}


History (loss, distributed):
	round 1: 0.05587423934936524
	round 2: 0.044970293760299686
	round 3: 0.04253035924434662
	round 4: 0.042628040552139285
	round 5: 0.044089994812011715
	round 6: 0.0470378776550293
	round 7: 0.05151897335052491
	round 8: 0.05631762299537659
	round 9: 0.062256294393539434
	round 10: 0.06569828534126282

 pid=8992) Client 3 loss 0.06818958950042725
 pid=8992) Client 3 accuracy 0.566


In [13]:
from typing import Callable, Union

from flwr.common import (
    EvaluateIns,
    EvaluateRes,
    FitIns,
    FitRes,
    MetricsAggregationFn,
    NDArrays,
    Parameters,
    Scalar,
    ndarrays_to_parameters,
    parameters_to_ndarrays,
)
from flwr.server.client_manager import ClientManager
from flwr.server.client_proxy import ClientProxy
from flwr.server.strategy.aggregate import aggregate, weighted_loss_avg


class FedCustom(fl.server.strategy.Strategy):
    def __init__(
        self,
        fraction_fit: float = 1.0,
        fraction_evaluate: float = 1.0,
        min_fit_clients: int = 2,
        min_evaluate_clients: int = 2,
        min_available_clients: int = 2,
    ) -> None:
        super().__init__()
        self.fraction_fit = fraction_fit
        self.fraction_evaluate = fraction_evaluate
        self.min_fit_clients = min_fit_clients
        self.min_evaluate_clients = min_evaluate_clients
        self.min_available_clients = min_available_clients

    def __repr__(self) -> str:
        return "FedCustom"

    def initialize_parameters(
        self, client_manager: ClientManager
    ) -> Optional[Parameters]:
        """Initialize global model parameters."""
        net = Net()
        ndarrays = get_parameters(net)
        return fl.common.ndarrays_to_parameters(ndarrays)

    def configure_fit(
        self, server_round: int, parameters: Parameters, client_manager: ClientManager
    ) -> List[Tuple[ClientProxy, FitIns]]:
        """Configure the next round of training."""

        # Sample clients
        sample_size, min_num_clients = self.num_fit_clients(
            client_manager.num_available()
        )
        clients = client_manager.sample(
            num_clients=sample_size, min_num_clients=min_num_clients
        )

        # Create custom configs
        n_clients = len(clients)
        #half_clients = n_clients // 2
        standard_config = {"lr": 0.05} #More Tighter LR
        higher_lr_config = {"lr": 0.003}
        fit_configurations = []
        for idx, client in enumerate(clients):
            fit_configurations.append((client, FitIns(parameters, standard_config)))
            # if idx < half_clients:
            #     fit_configurations.append((client, FitIns(parameters, standard_config)))
            # else:
            #     fit_configurations.append(
            #         (client, FitIns(parameters, higher_lr_config))
            #     )
        return fit_configurations

    def aggregate_fit(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, FitRes]],
        failures: List[Union[Tuple[ClientProxy, FitRes], BaseException]],
    ) -> Tuple[Optional[Parameters], Dict[str, Scalar]]:
        """Aggregate fit results using weighted average."""

        weights_results = [
            (parameters_to_ndarrays(fit_res.parameters), fit_res.num_examples)
            for _, fit_res in results
        ]
        parameters_aggregated = ndarrays_to_parameters(aggregate(weights_results))
        metrics_aggregated = {}
        return parameters_aggregated, metrics_aggregated

    def configure_evaluate(
        self, server_round: int, parameters: Parameters, client_manager: ClientManager
    ) -> List[Tuple[ClientProxy, EvaluateIns]]:
        """Configure the next round of evaluation."""
        if self.fraction_evaluate == 0.0:
            return []
        config = {}
        evaluate_ins = EvaluateIns(parameters, config)

        # Sample clients
        sample_size, min_num_clients = self.num_evaluation_clients(
            client_manager.num_available()
        )
        clients = client_manager.sample(
            num_clients=sample_size, min_num_clients=min_num_clients
        )

        # Return client/config pairs
        return [(client, evaluate_ins) for client in clients]

    def aggregate_evaluate(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, EvaluateRes]],
        failures: List[Union[Tuple[ClientProxy, EvaluateRes], BaseException]],
    ) -> Tuple[Optional[float], Dict[str, Scalar]]:
        """Aggregate evaluation losses using weighted average."""

        if not results:
            return None, {}

        loss_aggregated = weighted_loss_avg(
            [
                (evaluate_res.num_examples, evaluate_res.loss)
                for _, evaluate_res in results
            ]
        )
        metrics_aggregated = {}
        return loss_aggregated, metrics_aggregated

    def evaluate(
        self, server_round: int, parameters: Parameters
    ) -> Optional[Tuple[float, Dict[str, Scalar]]]:
        """Evaluate global model parameters using an evaluation function."""

        # Let's assume we won't perform the global model evaluation on the server side.
        return None

    def num_fit_clients(self, num_available_clients: int) -> Tuple[int, int]:
        """Return sample size and required number of clients."""
        num_clients = int(num_available_clients * self.fraction_fit)
        return max(num_clients, self.min_fit_clients), self.min_available_clients

    def num_evaluation_clients(self, num_available_clients: int) -> Tuple[int, int]:
        """Use a fraction of available clients for evaluation."""
        num_clients = int(num_available_clients * self.fraction_evaluate)
        return max(num_clients, self.min_evaluate_clients), self.min_available_clients

In [14]:
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=5,
    config=fl.server.ServerConfig(num_rounds=10),
    strategy=FedCustom(),  # <-- pass the new strategy here
    client_resources=client_resources,
)

INFO flwr 2024-04-26 10:57:51,363 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
INFO flwr 2024-04-26 10:57:57,246 | app.py:180 | Flower VCE: Ray initialized with resources: {'node:127.0.0.1': 1.0, 'memory': 65521953383.0, 'object_store_memory': 32366551449.0, 'CPU': 32.0, 'GPU': 1.0}
INFO flwr 2024-04-26 10:57:57,248 | server.py:86 | Initializing global parameters
INFO flwr 2024-04-26 10:57:57,254 | server.py:269 | Using initial parameters provided by strategy
INFO flwr 2024-04-26 10:57:57,256 | server.py:88 | Evaluating initial parameters
INFO flwr 2024-04-26 10:57:57,257 | server.py:101 | FL starting
DEBUG flwr 2024-04-26 10:57:57,258 | server.py:218 | fit_round 1: strategy sampled 5 clients (out of 5)


 pid=28072) [Client 3] fit, config: {'lr': 0.05}
 pid=28072) Epoch 1: train loss 0.06570011377334595, accuracy 0.23
 pid=28072) Epoch 2: train loss 0.05679967626929283, accuracy 0.33666666666666667
 pid=28072) Epoch 3: train loss 0.0518064871430397, accuracy 0.40244444444444444
 pid=28072) Epoch 4: train loss 0.04898172616958618, accuracy 0.4262222222222222
 pid=28072) Epoch 5: train loss 0.04759883135557175, accuracy 0.4528888888888889
 pid=28072) [Client 1] fit, config: {'lr': 0.05}
 pid=28072) Epoch 1: train loss 0.065456822514534, accuracy 0.2242222222222222
 pid=28072) Epoch 2: train loss 0.0551731213927269, accuracy 0.3473333333333333
 pid=28072) Epoch 3: train loss 0.05000154301524162, accuracy 0.416
 pid=28072) Epoch 4: train loss 0.047479771077632904, accuracy 0.44733333333333336
 pid=28072) Epoch 5: train loss 0.044916972517967224, accuracy 0.47688888888888886
 pid=28072) [Client 0] fit, config: {'lr': 0.05}
 pid=28072) Epoch 1: train loss 0.06508158147335052, accuracy 0.2328

DEBUG flwr 2024-04-26 10:59:22,838 | server.py:232 | fit_round 1 received 5 results and 0 failures
DEBUG flwr 2024-04-26 10:59:22,857 | server.py:168 | evaluate_round 1: strategy sampled 5 clients (out of 5)


 pid=28072) Epoch 5: train loss 0.04628932848572731, accuracy 0.46355555555555555
 pid=28072) [Client 4] evaluate, config: {}
 pid=28072) Client 4 loss 0.056903997898101806
 pid=28072) Client 4 accuracy 0.354
 pid=28072) [Client 0] evaluate, config: {}
 pid=28072) Client 0 loss 0.0573690927028656
 pid=28072) Client 0 accuracy 0.37
 pid=28072) [Client 3] evaluate, config: {}
 pid=28072) Client 3 loss 0.05719008088111877
 pid=28072) Client 3 accuracy 0.346
 pid=28072) [Client 1] evaluate, config: {}
 pid=28072) Client 1 loss 0.05709424734115601
 pid=28072) Client 1 accuracy 0.344
 pid=28072) [Client 2] evaluate, config: {}


DEBUG flwr 2024-04-26 10:59:30,636 | server.py:182 | evaluate_round 1 received 5 results and 0 failures
DEBUG flwr 2024-04-26 10:59:30,638 | server.py:218 | fit_round 2: strategy sampled 5 clients (out of 5)


...

DEBUG flwr 2024-04-26 11:11:39,348 | server.py:232 | fit_round 10 received 5 results and 0 failures
DEBUG flwr 2024-04-26 11:11:39,369 | server.py:168 | evaluate_round 10: strategy sampled 5 clients (out of 5)


 pid=28072) Epoch 5: train loss 0.005338515620678663, accuracy 0.9528888888888889
 pid=28072) [Client 4] evaluate, config: {}
 pid=28072) Client 4 loss 0.06450396728515626
 pid=28072) Client 4 accuracy 0.504
 pid=28072) [Client 0] evaluate, config: {}
 pid=28072) Client 0 loss 0.06822607636451722
 pid=28072) Client 0 accuracy 0.544
 pid=28072) [Client 3] evaluate, config: {}
 pid=28072) Client 3 loss 0.06748315238952637
 pid=28072) Client 3 accuracy 0.55
 pid=28072) [Client 1] evaluate, config: {}
 pid=28072) Client 1 loss 0.06841527080535889
 pid=28072) Client 1 accuracy 0.532
 pid=28072) [Client 2] evaluate, config: {}


DEBUG flwr 2024-04-26 11:11:47,192 | server.py:182 | evaluate_round 10 received 5 results and 0 failures
INFO flwr 2024-04-26 11:11:47,194 | server.py:147 | FL finished in 829.9360781999994
INFO flwr 2024-04-26 11:11:47,195 | app.py:218 | app_fit: losses_distributed [(1, 0.05734673700332642), (2, 0.046801169967651365), (3, 0.04356116471290589), (4, 0.04300533988475799), (5, 0.044804596757888795), (6, 0.04715286226272584), (7, 0.052886215138435354), (8, 0.05716916155815125), (9, 0.06228646445274353), (10, 0.06794561934471131)]
INFO flwr 2024-04-26 11:11:47,196 | app.py:219 | app_fit: metrics_distributed_fit {}
INFO flwr 2024-04-26 11:11:47,198 | app.py:220 | app_fit: metrics_distributed {}
INFO flwr 2024-04-26 11:11:47,199 | app.py:221 | app_fit: losses_centralized []
INFO flwr 2024-04-26 11:11:47,200 | app.py:222 | app_fit: metrics_centralized {}


 pid=28072) Client 2 loss 0.0710996298789978
 pid=28072) Client 2 accuracy 0.528


History (loss, distributed):
	round 1: 0.05734673700332642
	round 2: 0.046801169967651365
	round 3: 0.04356116471290589
	round 4: 0.04300533988475799
	round 5: 0.044804596757888795
	round 6: 0.04715286226272584
	round 7: 0.052886215138435354
	round 8: 0.05716916155815125
	round 9: 0.06228646445274353
	round 10: 0.06794561934471131

CPU ONLY FROM HERE

In [23]:
# Specify client resources if you need GPU (defaults to 1 CPU and 0 GPU)
client_resources = {"num_cpus":1}
if DEVICE.type == "cuda":
    client_resources = {"num_gpus": 1}

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=5,
    config=fl.server.ServerConfig(num_rounds=10),
    client_resources=client_resources,
)

INFO flwr 2024-04-26 16:55:42,045 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)


INFO flwr 2024-04-26 16:55:48,150 | app.py:180 | Flower VCE: Ray initialized with resources: {'CPU': 32.0, 'object_store_memory': 32586108518.0, 'memory': 66034253210.0, 'node:127.0.0.1': 1.0, 'GPU': 1.0}
INFO flwr 2024-04-26 16:55:48,153 | server.py:86 | Initializing global parameters
INFO flwr 2024-04-26 16:55:48,156 | server.py:273 | Requesting initial parameters from one random client
INFO flwr 2024-04-26 16:55:51,758 | server.py:277 | Received initial parameters from one random client
INFO flwr 2024-04-26 16:55:51,760 | server.py:88 | Evaluating initial parameters
INFO flwr 2024-04-26 16:55:51,761 | server.py:101 | FL starting
DEBUG flwr 2024-04-26 16:55:51,763 | server.py:218 | fit_round 1: strategy sampled 5 clients (out of 5)


 pid=30608) [Client 3] get_parameters
 pid=30608) [Client 0] fit, config: {}
 pid=24624) [Client 2] fit, config: {}
 pid=30148) [Client 1] fit, config: {}
 pid=30080) [Client 3] fit, config: {}
 pid=14704) [Client 4] fit, config: {}
 pid=30608) Epoch 1: train loss 0.06401211768388748, accuracy 0.24488888888888888
 pid=30148) Epoch 1: train loss 0.06456658244132996, accuracy 0.23933333333333334
 pid=30080) Epoch 1: train loss 0.06444425880908966, accuracy 0.23577777777777778
 pid=24624) Epoch 1: train loss 0.06404660642147064, accuracy 0.2408888888888889
 pid=14704) Epoch 1: train loss 0.06464178115129471, accuracy 0.23044444444444445
 pid=30608) Epoch 2: train loss 0.05591985210776329, accuracy 0.3486666666666667
 pid=30148) Epoch 2: train loss 0.05615413933992386, accuracy 0.33955555555555555
 pid=30080) Epoch 2: train loss 0.05566015839576721, accuracy 0.3446666666666667
 pid=24624) Epoch 2: train loss 0.05632326751947403, accuracy 0.3368888888888889
 pid=14704) Epoch 2: train loss 0

DEBUG flwr 2024-04-26 16:56:22,185 | server.py:232 | fit_round 1 received 5 results and 0 failures


 pid=24624) Epoch 5: train loss 0.047577500343322754, accuracy 0.438
 pid=30080) Epoch 5: train loss 0.04647189378738403, accuracy 0.4622222222222222
 pid=14704) Epoch 5: train loss 0.04621151462197304, accuracy 0.4651111111111111


...

DEBUG flwr 2024-04-26 17:01:39,635 | server.py:232 | fit_round 10 received 5 results and 0 failures
DEBUG flwr 2024-04-26 17:01:39,653 | server.py:168 | evaluate_round 10: strategy sampled 5 clients (out of 5)


 pid=24624) Epoch 5: train loss 0.005034103989601135, accuracy 0.9526666666666667
 pid=30080) [Client 3] evaluate, config: {}
 pid=24624) [Client 4] evaluate, config: {}
 pid=30608) [Client 0] evaluate, config: {}
 pid=30148) [Client 2] evaluate, config: {}
 pid=14704) [Client 1] evaluate, config: {}


DEBUG flwr 2024-04-26 17:01:46,898 | server.py:182 | evaluate_round 10 received 5 results and 0 failures
INFO flwr 2024-04-26 17:01:46,900 | server.py:147 | FL finished in 355.1370707999995
INFO flwr 2024-04-26 17:01:46,901 | app.py:218 | app_fit: losses_distributed [(1, 0.05439390110969543), (2, 0.045057359576225274), (3, 0.04282845077514648), (4, 0.04175490777492523), (5, 0.04348401856422425), (6, 0.04747948997020721), (7, 0.05145554873943329), (8, 0.057662916803359984), (9, 0.06256702167987824), (10, 0.06935367078781127)]
INFO flwr 2024-04-26 17:01:46,902 | app.py:219 | app_fit: metrics_distributed_fit {}
INFO flwr 2024-04-26 17:01:46,904 | app.py:220 | app_fit: metrics_distributed {}
INFO flwr 2024-04-26 17:01:46,905 | app.py:221 | app_fit: losses_centralized []
INFO flwr 2024-04-26 17:01:46,906 | app.py:222 | app_fit: metrics_centralized {}


History (loss, distributed):
	round 1: 0.05439390110969543
	round 2: 0.045057359576225274
	round 3: 0.04282845077514648
	round 4: 0.04175490777492523
	round 5: 0.04348401856422425
	round 6: 0.04747948997020721
	round 7: 0.05145554873943329
	round 8: 0.057662916803359984
	round 9: 0.06256702167987824
	round 10: 0.06935367078781127

 pid=30608) Client 0 loss 0.06547129249572754
 pid=30608) Client 0 accuracy 0.55
 pid=30148) Client 2 loss 0.06915074634552001
 pid=30148) Client 2 accuracy 0.548
 pid=30080) Client 3 loss 0.06717643356323243
 pid=30080) Client 3 accuracy 0.56
 pid=24624) Client 4 loss 0.07428802847862244
 pid=24624) Client 4 accuracy 0.5
 pid=14704) Client 1 loss 0.07068185305595398
 pid=14704) Client 1 accuracy 0.54


CPU 0.001

In [24]:
from typing import Callable, Union

from flwr.common import (
    EvaluateIns,
    EvaluateRes,
    FitIns,
    FitRes,
    MetricsAggregationFn,
    NDArrays,
    Parameters,
    Scalar,
    ndarrays_to_parameters,
    parameters_to_ndarrays,
)
from flwr.server.client_manager import ClientManager
from flwr.server.client_proxy import ClientProxy
from flwr.server.strategy.aggregate import aggregate, weighted_loss_avg


class FedCustom(fl.server.strategy.Strategy):
    def __init__(
        self,
        fraction_fit: float = 1.0,
        fraction_evaluate: float = 1.0,
        min_fit_clients: int = 2,
        min_evaluate_clients: int = 2,
        min_available_clients: int = 2,
    ) -> None:
        super().__init__()
        self.fraction_fit = fraction_fit
        self.fraction_evaluate = fraction_evaluate
        self.min_fit_clients = min_fit_clients
        self.min_evaluate_clients = min_evaluate_clients
        self.min_available_clients = min_available_clients

    def __repr__(self) -> str:
        return "FedCustom"

    def initialize_parameters(
        self, client_manager: ClientManager
    ) -> Optional[Parameters]:
        """Initialize global model parameters."""
        net = Net()
        ndarrays = get_parameters(net)
        return fl.common.ndarrays_to_parameters(ndarrays)

    def configure_fit(
        self, server_round: int, parameters: Parameters, client_manager: ClientManager
    ) -> List[Tuple[ClientProxy, FitIns]]:
        """Configure the next round of training."""

        # Sample clients
        sample_size, min_num_clients = self.num_fit_clients(
            client_manager.num_available()
        )
        clients = client_manager.sample(
            num_clients=sample_size, min_num_clients=min_num_clients
        )

        # Create custom configs
        n_clients = len(clients)
        #half_clients = n_clients // 2
        standard_config = {"lr": 0.001}
        higher_lr_config = {"lr": 0.003}
        fit_configurations = []
        for idx, client in enumerate(clients):
            fit_configurations.append((client, FitIns(parameters, standard_config)))
            # if idx < half_clients:
            #     fit_configurations.append((client, FitIns(parameters, standard_config)))
            # else:
            #     fit_configurations.append(
            #         (client, FitIns(parameters, higher_lr_config))
            #     )
        return fit_configurations

    def aggregate_fit(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, FitRes]],
        failures: List[Union[Tuple[ClientProxy, FitRes], BaseException]],
    ) -> Tuple[Optional[Parameters], Dict[str, Scalar]]:
        """Aggregate fit results using weighted average."""

        weights_results = [
            (parameters_to_ndarrays(fit_res.parameters), fit_res.num_examples)
            for _, fit_res in results
        ]
        parameters_aggregated = ndarrays_to_parameters(aggregate(weights_results))
        metrics_aggregated = {}
        return parameters_aggregated, metrics_aggregated

    def configure_evaluate(
        self, server_round: int, parameters: Parameters, client_manager: ClientManager
    ) -> List[Tuple[ClientProxy, EvaluateIns]]:
        """Configure the next round of evaluation."""
        if self.fraction_evaluate == 0.0:
            return []
        config = {}
        evaluate_ins = EvaluateIns(parameters, config)

        # Sample clients
        sample_size, min_num_clients = self.num_evaluation_clients(
            client_manager.num_available()
        )
        clients = client_manager.sample(
            num_clients=sample_size, min_num_clients=min_num_clients
        )

        # Return client/config pairs
        return [(client, evaluate_ins) for client in clients]

    def aggregate_evaluate(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, EvaluateRes]],
        failures: List[Union[Tuple[ClientProxy, EvaluateRes], BaseException]],
    ) -> Tuple[Optional[float], Dict[str, Scalar]]:
        """Aggregate evaluation losses using weighted average."""

        if not results:
            return None, {}

        loss_aggregated = weighted_loss_avg(
            [
                (evaluate_res.num_examples, evaluate_res.loss)
                for _, evaluate_res in results
            ]
        )
        metrics_aggregated = {}
        return loss_aggregated, metrics_aggregated

    def evaluate(
        self, server_round: int, parameters: Parameters
    ) -> Optional[Tuple[float, Dict[str, Scalar]]]:
        """Evaluate global model parameters using an evaluation function."""

        # Let's assume we won't perform the global model evaluation on the server side.
        return None

    def num_fit_clients(self, num_available_clients: int) -> Tuple[int, int]:
        """Return sample size and required number of clients."""
        num_clients = int(num_available_clients * self.fraction_fit)
        return max(num_clients, self.min_fit_clients), self.min_available_clients

    def num_evaluation_clients(self, num_available_clients: int) -> Tuple[int, int]:
        """Use a fraction of available clients for evaluation."""
        num_clients = int(num_available_clients * self.fraction_evaluate)
        return max(num_clients, self.min_evaluate_clients), self.min_available_clients

In [25]:
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=5,
    config=fl.server.ServerConfig(num_rounds=10),
    strategy=FedCustom(),  # <-- pass the new strategy here
    client_resources=client_resources,
)

INFO flwr 2024-04-26 17:01:47,043 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
INFO flwr 2024-04-26 17:02:01,809 | app.py:180 | Flower VCE: Ray initialized with resources: {'memory': 66159352013.0, 'CPU': 32.0, 'GPU': 1.0, 'node:127.0.0.1': 1.0, 'object_store_memory': 32639722291.0}
INFO flwr 2024-04-26 17:02:01,812 | server.py:86 | Initializing global parameters
INFO flwr 2024-04-26 17:02:01,822 | server.py:269 | Using initial parameters provided by strategy
INFO flwr 2024-04-26 17:02:01,823 | server.py:88 | Evaluating initial parameters
INFO flwr 2024-04-26 17:02:01,825 | server.py:101 | FL starting
DEBUG flwr 2024-04-26 17:02:01,826 | server.py:218 | fit_round 1: strategy sampled 5 clients (out of 5)


 pid=15072) [Client 4] fit, config: {'lr': 0.001}
 pid=29136) [Client 3] fit, config: {'lr': 0.001}
 pid=24244) [Client 2] fit, config: {'lr': 0.001}
 pid=27924) [Client 1] fit, config: {'lr': 0.001}
 pid=10004) [Client 0] fit, config: {'lr': 0.001}
 pid=15072) Epoch 1: train loss 0.06498923897743225, accuracy 0.2117777777777778
 pid=10004) Epoch 1: train loss 0.06474829465150833, accuracy 0.2288888888888889
 pid=29136) Epoch 1: train loss 0.0650850459933281, accuracy 0.21977777777777777
 pid=24244) Epoch 1: train loss 0.06506025046110153, accuracy 0.21133333333333335
 pid=27924) Epoch 1: train loss 0.06447216123342514, accuracy 0.23333333333333334
 pid=10004) Epoch 2: train loss 0.05709301680326462, accuracy 0.33044444444444443
 pid=24244) Epoch 2: train loss 0.057777851819992065, accuracy 0.3148888888888889
 pid=15072) Epoch 2: train loss 0.05659504979848862, accuracy 0.33266666666666667
 pid=29136) Epoch 2: train loss 0.057149454951286316, accuracy 0.31577777777777777
 pid=27924) Ep

DEBUG flwr 2024-04-26 17:02:32,173 | server.py:232 | fit_round 1 received 5 results and 0 failures
DEBUG flwr 2024-04-26 17:02:32,199 | server.py:168 | evaluate_round 1: strategy sampled 5 clients (out of 5)


 pid=24244) Epoch 5: train loss 0.04779421538114548, accuracy 0.4408888888888889
 pid=27924) Epoch 5: train loss 0.047023821622133255, accuracy 0.4508888888888889
 pid=27924) [Client 0] evaluate, config: {}
 pid=29136) [Client 1] evaluate, config: {}
 pid=24244) [Client 2] evaluate, config: {}
 pid=15072) [Client 4] evaluate, config: {}
 pid=10004) [Client 3] evaluate, config: {}


DEBUG flwr 2024-04-26 17:02:39,589 | server.py:182 | evaluate_round 1 received 5 results and 0 failures
DEBUG flwr 2024-04-26 17:02:39,591 | server.py:218 | fit_round 2: strategy sampled 5 clients (out of 5)


...

 pid=15072) Epoch 5: train loss 0.006032404489815235, accuracy 0.942
 pid=15072) [Client 4] evaluate, config: {}
 pid=15072) Client 4 loss 0.06479401659965515
 pid=15072) Client 4 accuracy 0.536
 pid=29136) [Client 3] evaluate, config: {}
 pid=15072) [Client 1] evaluate, config: {}
 pid=10004) [Client 2] evaluate, config: {}


DEBUG flwr 2024-04-26 17:07:55,809 | server.py:182 | evaluate_round 10 received 5 results and 0 failures
INFO flwr 2024-04-26 17:07:55,811 | server.py:147 | FL finished in 353.98450720000255
INFO flwr 2024-04-26 17:07:55,812 | app.py:218 | app_fit: losses_distributed [(1, 0.05526458411216736), (2, 0.04518063118457794), (3, 0.04336182243824005), (4, 0.042881158232688905), (5, 0.043915944933891296), (6, 0.04619976933002472), (7, 0.05137586009502411), (8, 0.05645895371437073), (9, 0.06136216406822205), (10, 0.06766163568496704)]
INFO flwr 2024-04-26 17:07:55,814 | app.py:219 | app_fit: metrics_distributed_fit {}
INFO flwr 2024-04-26 17:07:55,815 | app.py:220 | app_fit: metrics_distributed {}
INFO flwr 2024-04-26 17:07:55,816 | app.py:221 | app_fit: losses_centralized []
INFO flwr 2024-04-26 17:07:55,817 | app.py:222 | app_fit: metrics_centralized {}


 pid=24244) [Client 0] evaluate, config: {}
 pid=15072) Client 1 loss 0.06667416024208069
 pid=15072) Client 1 accuracy 0.54
 pid=29136) Client 3 loss 0.06476444959640502
 pid=29136) Client 3 accuracy 0.554


History (loss, distributed):
	round 1: 0.05526458411216736
	round 2: 0.04518063118457794
	round 3: 0.04336182243824005
	round 4: 0.042881158232688905
	round 5: 0.043915944933891296
	round 6: 0.04619976933002472
	round 7: 0.05137586009502411
	round 8: 0.05645895371437073
	round 9: 0.06136216406822205
	round 10: 0.06766163568496704

 pid=24244) Client 0 loss 0.07001027870178222
 pid=24244) Client 0 accuracy 0.516
 pid=10004) Client 2 loss 0.0720652732849121
 pid=10004) Client 2 accuracy 0.524


In [26]:
from typing import Callable, Union

from flwr.common import (
    EvaluateIns,
    EvaluateRes,
    FitIns,
    FitRes,
    MetricsAggregationFn,
    NDArrays,
    Parameters,
    Scalar,
    ndarrays_to_parameters,
    parameters_to_ndarrays,
)
from flwr.server.client_manager import ClientManager
from flwr.server.client_proxy import ClientProxy
from flwr.server.strategy.aggregate import aggregate, weighted_loss_avg


class FedCustom(fl.server.strategy.Strategy):
    def __init__(
        self,
        fraction_fit: float = 1.0,
        fraction_evaluate: float = 1.0,
        min_fit_clients: int = 2,
        min_evaluate_clients: int = 2,
        min_available_clients: int = 2,
    ) -> None:
        super().__init__()
        self.fraction_fit = fraction_fit
        self.fraction_evaluate = fraction_evaluate
        self.min_fit_clients = min_fit_clients
        self.min_evaluate_clients = min_evaluate_clients
        self.min_available_clients = min_available_clients

    def __repr__(self) -> str:
        return "FedCustom"

    def initialize_parameters(
        self, client_manager: ClientManager
    ) -> Optional[Parameters]:
        """Initialize global model parameters."""
        net = Net()
        ndarrays = get_parameters(net)
        return fl.common.ndarrays_to_parameters(ndarrays)

    def configure_fit(
        self, server_round: int, parameters: Parameters, client_manager: ClientManager
    ) -> List[Tuple[ClientProxy, FitIns]]:
        """Configure the next round of training."""

        # Sample clients
        sample_size, min_num_clients = self.num_fit_clients(
            client_manager.num_available()
        )
        clients = client_manager.sample(
            num_clients=sample_size, min_num_clients=min_num_clients
        )

        # Create custom configs
        n_clients = len(clients)
        #half_clients = n_clients // 2
        standard_config = {"lr": 0.0001}
        higher_lr_config = {"lr": 0.003}
        fit_configurations = []
        for idx, client in enumerate(clients):
            fit_configurations.append((client, FitIns(parameters, standard_config)))
            # if idx < half_clients:
            #     fit_configurations.append((client, FitIns(parameters, standard_config)))
            # else:
            #     fit_configurations.append(
            #         (client, FitIns(parameters, higher_lr_config))
            #     )
        return fit_configurations

    def aggregate_fit(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, FitRes]],
        failures: List[Union[Tuple[ClientProxy, FitRes], BaseException]],
    ) -> Tuple[Optional[Parameters], Dict[str, Scalar]]:
        """Aggregate fit results using weighted average."""

        weights_results = [
            (parameters_to_ndarrays(fit_res.parameters), fit_res.num_examples)
            for _, fit_res in results
        ]
        parameters_aggregated = ndarrays_to_parameters(aggregate(weights_results))
        metrics_aggregated = {}
        return parameters_aggregated, metrics_aggregated

    def configure_evaluate(
        self, server_round: int, parameters: Parameters, client_manager: ClientManager
    ) -> List[Tuple[ClientProxy, EvaluateIns]]:
        """Configure the next round of evaluation."""
        if self.fraction_evaluate == 0.0:
            return []
        config = {}
        evaluate_ins = EvaluateIns(parameters, config)

        # Sample clients
        sample_size, min_num_clients = self.num_evaluation_clients(
            client_manager.num_available()
        )
        clients = client_manager.sample(
            num_clients=sample_size, min_num_clients=min_num_clients
        )

        # Return client/config pairs
        return [(client, evaluate_ins) for client in clients]

    def aggregate_evaluate(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, EvaluateRes]],
        failures: List[Union[Tuple[ClientProxy, EvaluateRes], BaseException]],
    ) -> Tuple[Optional[float], Dict[str, Scalar]]:
        """Aggregate evaluation losses using weighted average."""

        if not results:
            return None, {}

        loss_aggregated = weighted_loss_avg(
            [
                (evaluate_res.num_examples, evaluate_res.loss)
                for _, evaluate_res in results
            ]
        )
        metrics_aggregated = {}
        return loss_aggregated, metrics_aggregated

    def evaluate(
        self, server_round: int, parameters: Parameters
    ) -> Optional[Tuple[float, Dict[str, Scalar]]]:
        """Evaluate global model parameters using an evaluation function."""

        # Let's assume we won't perform the global model evaluation on the server side.
        return None

    def num_fit_clients(self, num_available_clients: int) -> Tuple[int, int]:
        """Return sample size and required number of clients."""
        num_clients = int(num_available_clients * self.fraction_fit)
        return max(num_clients, self.min_fit_clients), self.min_available_clients

    def num_evaluation_clients(self, num_available_clients: int) -> Tuple[int, int]:
        """Use a fraction of available clients for evaluation."""
        num_clients = int(num_available_clients * self.fraction_evaluate)
        return max(num_clients, self.min_evaluate_clients), self.min_available_clients

In [27]:
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=5,
    config=fl.server.ServerConfig(num_rounds=10),
    strategy=FedCustom(),  # <-- pass the new strategy here
    client_resources=client_resources,
)

INFO flwr 2024-04-26 17:07:55,944 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
INFO flwr 2024-04-26 17:08:10,551 | app.py:180 | Flower VCE: Ray initialized with resources: {'CPU': 32.0, 'object_store_memory': 32633033932.0, 'memory': 66143745844.0, 'node:127.0.0.1': 1.0, 'GPU': 1.0}
INFO flwr 2024-04-26 17:08:10,554 | server.py:86 | Initializing global parameters
INFO flwr 2024-04-26 17:08:10,564 | server.py:269 | Using initial parameters provided by strategy
INFO flwr 2024-04-26 17:08:10,566 | server.py:88 | Evaluating initial parameters
INFO flwr 2024-04-26 17:08:10,568 | server.py:101 | FL starting
DEBUG flwr 2024-04-26 17:08:10,570 | server.py:218 | fit_round 1: strategy sampled 5 clients (out of 5)


 pid=28516) [Client 2] fit, config: {'lr': 0.0001}
 pid=22372) [Client 3] fit, config: {'lr': 0.0001}
 pid=24600) [Client 0] fit, config: {'lr': 0.0001}
 pid=30204) [Client 4] fit, config: {'lr': 0.0001}
 pid=27788) [Client 1] fit, config: {'lr': 0.0001}
 pid=28516) Epoch 1: train loss 0.06471177190542221, accuracy 0.23777777777777778
 pid=22372) Epoch 1: train loss 0.06432643532752991, accuracy 0.2331111111111111
 pid=24600) Epoch 1: train loss 0.06451523303985596, accuracy 0.23955555555555555
 pid=30204) Epoch 1: train loss 0.06430403143167496, accuracy 0.2288888888888889
 pid=27788) Epoch 1: train loss 0.06504929810762405, accuracy 0.22111111111111112
 pid=28516) Epoch 2: train loss 0.05569467321038246, accuracy 0.35088888888888886
 pid=22372) Epoch 2: train loss 0.05553014948964119, accuracy 0.34644444444444444
 pid=30204) Epoch 2: train loss 0.05503148213028908, accuracy 0.35777777777777775
 pid=27788) Epoch 2: train loss 0.05618993565440178, accuracy 0.3466666666666667
 pid=24600

DEBUG flwr 2024-04-26 17:08:40,674 | server.py:232 | fit_round 1 received 5 results and 0 failures
DEBUG flwr 2024-04-26 17:08:40,725 | server.py:168 | evaluate_round 1: strategy sampled 5 clients (out of 5)


 pid=30204) Epoch 5: train loss 0.04526793956756592, accuracy 0.47688888888888886
 pid=24600) Epoch 5: train loss 0.045819688588380814, accuracy 0.46066666666666667
 pid=24600) [Client 3] evaluate, config: {}
 pid=30204) [Client 4] evaluate, config: {}
 pid=24600) Client 3 loss 0.05482020664215088
 pid=24600) Client 3 accuracy 0.376
 pid=30204) Client 4 loss 0.05357328152656555
 pid=30204) Client 4 accuracy 0.416
 pid=30204) [Client 2] evaluate, config: {}
 pid=24600) [Client 1] evaluate, config: {}


DEBUG flwr 2024-04-26 17:08:47,777 | server.py:182 | evaluate_round 1 received 5 results and 0 failures
DEBUG flwr 2024-04-26 17:08:47,779 | server.py:218 | fit_round 2: strategy sampled 5 clients (out of 5)


...

DEBUG flwr 2024-04-26 17:13:56,240 | server.py:232 | fit_round 10 received 5 results and 0 failures
DEBUG flwr 2024-04-26 17:13:56,258 | server.py:168 | evaluate_round 10: strategy sampled 5 clients (out of 5)


 pid=30204) Epoch 5: train loss 0.0041454853489995, accuracy 0.9608888888888889
 pid=28516) [Client 2] evaluate, config: {}
 pid=22372) [Client 4] evaluate, config: {}
 pid=24600) [Client 1] evaluate, config: {}
 pid=30204) [Client 3] evaluate, config: {}
 pid=27788) [Client 0] evaluate, config: {}


DEBUG flwr 2024-04-26 17:14:03,485 | server.py:182 | evaluate_round 10 received 5 results and 0 failures
INFO flwr 2024-04-26 17:14:03,487 | server.py:147 | FL finished in 352.9172909000008
INFO flwr 2024-04-26 17:14:03,488 | app.py:218 | app_fit: losses_distributed [(1, 0.05482563681602478), (2, 0.04504301114082336), (3, 0.04316891150474548), (4, 0.042675419616699226), (5, 0.04495172107219696), (6, 0.04859298067092895), (7, 0.054215183949470516), (8, 0.06054967880249024), (9, 0.06760601172447205), (10, 0.07175511364936829)]
INFO flwr 2024-04-26 17:14:03,489 | app.py:219 | app_fit: metrics_distributed_fit {}
INFO flwr 2024-04-26 17:14:03,491 | app.py:220 | app_fit: metrics_distributed {}
INFO flwr 2024-04-26 17:14:03,492 | app.py:221 | app_fit: losses_centralized []
INFO flwr 2024-04-26 17:14:03,493 | app.py:222 | app_fit: metrics_centralized {}


History (loss, distributed):
	round 1: 0.05482563681602478
	round 2: 0.04504301114082336
	round 3: 0.04316891150474548
	round 4: 0.042675419616699226
	round 5: 0.04495172107219696
	round 6: 0.04859298067092895
	round 7: 0.054215183949470516
	round 8: 0.06054967880249024
	round 9: 0.06760601172447205
	round 10: 0.07175511364936829

 pid=28516) Client 2 loss 0.07303936862945556
 pid=28516) Client 2 accuracy 0.532
 pid=22372) Client 4 loss 0.07555317211151123
 pid=22372) Client 4 accuracy 0.54
 pid=24600) Client 1 loss 0.07152706336975098
 pid=24600) Client 1 accuracy 0.56
 pid=30204) Client 3 loss 0.06725288581848145
 pid=30204) Client 3 accuracy 0.562
 pid=27788) Client 0 loss 0.07140307831764221
 pid=27788) Client 0 accuracy 0.554


In [28]:
from typing import Callable, Union

from flwr.common import (
    EvaluateIns,
    EvaluateRes,
    FitIns,
    FitRes,
    MetricsAggregationFn,
    NDArrays,
    Parameters,
    Scalar,
    ndarrays_to_parameters,
    parameters_to_ndarrays,
)
from flwr.server.client_manager import ClientManager
from flwr.server.client_proxy import ClientProxy
from flwr.server.strategy.aggregate import aggregate, weighted_loss_avg


class FedCustom(fl.server.strategy.Strategy):
    def __init__(
        self,
        fraction_fit: float = 1.0,
        fraction_evaluate: float = 1.0,
        min_fit_clients: int = 2,
        min_evaluate_clients: int = 2,
        min_available_clients: int = 2,
    ) -> None:
        super().__init__()
        self.fraction_fit = fraction_fit
        self.fraction_evaluate = fraction_evaluate
        self.min_fit_clients = min_fit_clients
        self.min_evaluate_clients = min_evaluate_clients
        self.min_available_clients = min_available_clients

    def __repr__(self) -> str:
        return "FedCustom"

    def initialize_parameters(
        self, client_manager: ClientManager
    ) -> Optional[Parameters]:
        """Initialize global model parameters."""
        net = Net()
        ndarrays = get_parameters(net)
        return fl.common.ndarrays_to_parameters(ndarrays)

    def configure_fit(
        self, server_round: int, parameters: Parameters, client_manager: ClientManager
    ) -> List[Tuple[ClientProxy, FitIns]]:
        """Configure the next round of training."""

        # Sample clients
        sample_size, min_num_clients = self.num_fit_clients(
            client_manager.num_available()
        )
        clients = client_manager.sample(
            num_clients=sample_size, min_num_clients=min_num_clients
        )

        # Create custom configs
        n_clients = len(clients)
        #half_clients = n_clients // 2
        standard_config = {"lr": 0.01}
        higher_lr_config = {"lr": 0.003}
        fit_configurations = []
        for idx, client in enumerate(clients):
            fit_configurations.append((client, FitIns(parameters, standard_config)))
            # if idx < half_clients:
            #     fit_configurations.append((client, FitIns(parameters, standard_config)))
            # else:
            #     fit_configurations.append(
            #         (client, FitIns(parameters, higher_lr_config))
            #     )
        return fit_configurations

    def aggregate_fit(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, FitRes]],
        failures: List[Union[Tuple[ClientProxy, FitRes], BaseException]],
    ) -> Tuple[Optional[Parameters], Dict[str, Scalar]]:
        """Aggregate fit results using weighted average."""

        weights_results = [
            (parameters_to_ndarrays(fit_res.parameters), fit_res.num_examples)
            for _, fit_res in results
        ]
        parameters_aggregated = ndarrays_to_parameters(aggregate(weights_results))
        metrics_aggregated = {}
        return parameters_aggregated, metrics_aggregated

    def configure_evaluate(
        self, server_round: int, parameters: Parameters, client_manager: ClientManager
    ) -> List[Tuple[ClientProxy, EvaluateIns]]:
        """Configure the next round of evaluation."""
        if self.fraction_evaluate == 0.0:
            return []
        config = {}
        evaluate_ins = EvaluateIns(parameters, config)

        # Sample clients
        sample_size, min_num_clients = self.num_evaluation_clients(
            client_manager.num_available()
        )
        clients = client_manager.sample(
            num_clients=sample_size, min_num_clients=min_num_clients
        )

        # Return client/config pairs
        return [(client, evaluate_ins) for client in clients]

    def aggregate_evaluate(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, EvaluateRes]],
        failures: List[Union[Tuple[ClientProxy, EvaluateRes], BaseException]],
    ) -> Tuple[Optional[float], Dict[str, Scalar]]:
        """Aggregate evaluation losses using weighted average."""

        if not results:
            return None, {}

        loss_aggregated = weighted_loss_avg(
            [
                (evaluate_res.num_examples, evaluate_res.loss)
                for _, evaluate_res in results
            ]
        )
        metrics_aggregated = {}
        return loss_aggregated, metrics_aggregated

    def evaluate(
        self, server_round: int, parameters: Parameters
    ) -> Optional[Tuple[float, Dict[str, Scalar]]]:
        """Evaluate global model parameters using an evaluation function."""

        # Let's assume we won't perform the global model evaluation on the server side.
        return None

    def num_fit_clients(self, num_available_clients: int) -> Tuple[int, int]:
        """Return sample size and required number of clients."""
        num_clients = int(num_available_clients * self.fraction_fit)
        return max(num_clients, self.min_fit_clients), self.min_available_clients

    def num_evaluation_clients(self, num_available_clients: int) -> Tuple[int, int]:
        """Use a fraction of available clients for evaluation."""
        num_clients = int(num_available_clients * self.fraction_evaluate)
        return max(num_clients, self.min_evaluate_clients), self.min_available_clients

In [29]:
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=5,
    config=fl.server.ServerConfig(num_rounds=10),
    strategy=FedCustom(),  # <-- pass the new strategy here
    client_resources=client_resources,
)

INFO flwr 2024-04-26 17:14:03,631 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
INFO flwr 2024-04-26 17:14:18,321 | app.py:180 | Flower VCE: Ray initialized with resources: {'object_store_memory': 32606579097.0, 'memory': 66082017895.0, 'GPU': 1.0, 'CPU': 32.0, 'node:127.0.0.1': 1.0}
INFO flwr 2024-04-26 17:14:18,323 | server.py:86 | Initializing global parameters
INFO flwr 2024-04-26 17:14:18,336 | server.py:269 | Using initial parameters provided by strategy
INFO flwr 2024-04-26 17:14:18,338 | server.py:88 | Evaluating initial parameters
INFO flwr 2024-04-26 17:14:18,340 | server.py:101 | FL starting
DEBUG flwr 2024-04-26 17:14:18,342 | server.py:218 | fit_round 1: strategy sampled 5 clients (out of 5)


 pid=26480) [Client 0] fit, config: {'lr': 0.01}
 pid=30584) [Client 3] fit, config: {'lr': 0.01}
 pid=25168) [Client 2] fit, config: {'lr': 0.01}
 pid=24688) [Client 1] fit, config: {'lr': 0.01}
 pid=26992) [Client 4] fit, config: {'lr': 0.01}
 pid=26480) Epoch 1: train loss 0.06421691924333572, accuracy 0.23577777777777778
 pid=24688) Epoch 1: train loss 0.06505580991506577, accuracy 0.22177777777777777
 pid=26992) Epoch 1: train loss 0.06500303745269775, accuracy 0.2157777777777778
 pid=25168) Epoch 1: train loss 0.06502891331911087, accuracy 0.23244444444444445
 pid=30584) Epoch 1: train loss 0.0655265524983406, accuracy 0.21088888888888888
 pid=26480) Epoch 2: train loss 0.05542409420013428, accuracy 0.3531111111111111
 pid=26992) Epoch 2: train loss 0.057087816298007965, accuracy 0.33355555555555555
 pid=24688) Epoch 2: train loss 0.05554568022489548, accuracy 0.34844444444444445
 pid=25168) Epoch 2: train loss 0.05750065669417381, accuracy 0.3342222222222222
 pid=30584) Epoch 2:

DEBUG flwr 2024-04-26 17:14:48,527 | server.py:232 | fit_round 1 received 5 results and 0 failures


 pid=26480) Epoch 5: train loss 0.04633060842752457, accuracy 0.4568888888888889
 pid=25168) Epoch 5: train loss 0.04754297807812691, accuracy 0.4351111111111111
 pid=30584) Epoch 5: train loss 0.047654155641794205, accuracy 0.43822222222222224
 pid=24688) Epoch 5: train loss 0.04643639922142029, accuracy 0.45222222222222225


DEBUG flwr 2024-04-26 17:14:48,568 | server.py:168 | evaluate_round 1: strategy sampled 5 clients (out of 5)


...

DEBUG flwr 2024-04-26 17:20:12,737 | server.py:182 | evaluate_round 10 received 5 results and 0 failures


 pid=26480) Client 0 loss 0.06004209458827972
 pid=26480) Client 0 accuracy 0.578


INFO flwr 2024-04-26 17:20:12,739 | server.py:147 | FL finished in 354.3971630999986
INFO flwr 2024-04-26 17:20:12,741 | app.py:218 | app_fit: losses_distributed [(1, 0.05780576496124268), (2, 0.04547640619277954), (3, 0.04203104388713837), (4, 0.04069131336212158), (5, 0.04216132199764251), (6, 0.04342163672447205), (7, 0.04793027019500733), (8, 0.05186192860603332), (9, 0.056686721897125245), (10, 0.06290451562404634)]
INFO flwr 2024-04-26 17:20:12,742 | app.py:219 | app_fit: metrics_distributed_fit {}
INFO flwr 2024-04-26 17:20:12,744 | app.py:220 | app_fit: metrics_distributed {}
INFO flwr 2024-04-26 17:20:12,746 | app.py:221 | app_fit: losses_centralized []
INFO flwr 2024-04-26 17:20:12,747 | app.py:222 | app_fit: metrics_centralized {}


History (loss, distributed):
	round 1: 0.05780576496124268
	round 2: 0.04547640619277954
	round 3: 0.04203104388713837
	round 4: 0.04069131336212158
	round 5: 0.04216132199764251
	round 6: 0.04342163672447205
	round 7: 0.04793027019500733
	round 8: 0.05186192860603332
	round 9: 0.056686721897125245
	round 10: 0.06290451562404634

 pid=26992) Client 3 loss 0.06129781699180603
 pid=26992) Client 3 accuracy 0.594
 pid=24688) Client 4 loss 0.0623255615234375
 pid=24688) Client 4 accuracy 0.516
 pid=25168) Client 2 loss 0.06739232540130616
 pid=25168) Client 2 accuracy 0.534


## Recap

In this notebook, we’ve seen how to implement a custom strategy. A custom strategy enables granular control over client node configuration, result aggregation, and more. To define a custom strategy, you only have to overwrite the abstract methods of the (abstract) base class `Strategy`. To make custom strategies even more powerful, you can pass custom functions to the constructor of your new class (`__init__`) and then call these functions whenever needed.

## Next steps

Before you continue, make sure to join the Flower community on Slack: [Join Slack](https://flower.ai/join-slack/)

There's a dedicated `#questions` channel if you need help, but we'd also love to hear who you are in `#introductions`!

The [Flower Federated Learning Tutorial - Part 4](https://flower.ai/docs/framework/tutorial-customize-the-client-pytorch.html) introduces `Client`, the flexible API underlying `NumPyClient`.